<a href="https://colab.research.google.com/github/HuuPhuoc2411/ESP32Feature28/blob/main/train_toi_uu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Google Colab Machine Learning Pipeline for SVM Models
Automatically train SVM models and generate Arduino/ESP32 libraries
Author: Auto-generated
Date: 2025
"""

# ============================================================================
# INSTALL REQUIRED PACKAGES (For Google Colab)
# ============================================================================
import sys

# Kiểm tra xem có đang chạy trên Colab không
try:
    import google.colab
    IN_COLAB_CHECK = True
except:
    IN_COLAB_CHECK = False

# Kiểm tra xem có đang chạy trên Kaggle không
try:
    import kaggle
    IN_KAGGLE_CHECK = True
except:
    IN_KAGGLE_CHECK = False

# Cài đặt các thư viện cần thiết
if IN_COLAB_CHECK or IN_KAGGLE_CHECK:
    print("📦 Đang cài đặt: optuna, openpyxl, gradio...")
    !pip install -q optuna openpyxl gradio
    print("✓ Cài đặt hoàn tất!")

# ============================================================================
# IMPORT LIBRARIES
# ============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.svm import SVC, NuSVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier,
                               GradientBoostingClassifier, AdaBoostClassifier)
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, confusion_matrix, classification_report)
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner
import json
import os
import time
import pickle
import shutil
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=optuna.exceptions.ExperimentalWarning)

# For Google Colab
try:
    from google.colab import files, drive
    IN_COLAB = True
except:
    IN_COLAB = False

# For Kaggle
try:
    import kaggle
    IN_KAGGLE = True
except:
    IN_KAGGLE = False

# Check if running in cloud environment
IN_CLOUD = IN_COLAB or IN_KAGGLE

# Import Gradio cho giao diện web
try:
    import gradio as gr
    GRADIO_AVAILABLE = True
except ImportError:
    GRADIO_AVAILABLE = False
    if not IN_CLOUD:
        print("⚠️  Gradio chưa được cài đặt. Chạy: pip install gradio")

# ============================================================================
# CONFIGURATION - NGƯỜI DÙNG TỰ CẤU HÌNH Ở ĐÂY
# ============================================================================

# --- Đường dẫn file (chỉ dùng khi chạy CLI mode, Gradio mode tải qua giao diện) ---
SINGLE_FILE_PATH = ""  # Để trống nếu dùng Gradio
TRAIN_FILE_PATH = ""   # Để trống nếu dùng Gradio
TEST_FILE_PATH = ""    # Để trống nếu dùng Gradio

# --- Cấu hình chia dữ liệu ---
TRAIN_RATIO = 0.7  # Tỷ lệ train (70%), test sẽ là 30%
RANDOM_STATE = 42

# --- Cấu hình Optuna ---
N_TRIALS = 50  # Số lần thử tối ưu hóa (tăng lên để kết quả tốt hơn nhưng mất thời gian)
CV_FOLDS = 5   # Số fold cho cross-validation
OPTUNA_TIMEOUT = 180  # Timeout cho mỗi model (giây) - giảm xuống để train nhanh hơn
MIN_TRIALS_BEFORE_PRUNING = 3  # Số trial tối thiểu trước khi bắt đầu prune
PRUNING_WARMUP_STEPS = 1  # Số fold tối thiểu trước khi prune trial

# --- Cấu hình output ---
OUTPUT_DIR = "ML_Output"  # Thư mục chứa kết quả
ARDUINO_LIB_NAME = "MLPredictor"  # Tên thư viện Arduino
SEPARATE_MODEL_FILES = True  # True: Mỗi model 1 file riêng (tối ưu ROM), False: Tất cả trong 1 file

# ============================================================================
# SPXY ALGORITHM - KENNARD-STONE ALGORITHM
# ============================================================================

def calculate_distance_matrix(X):
    """Tính ma trận khoảng cách Euclidean"""
    n = X.shape[0]
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            dist = np.linalg.norm(X[i] - X[j])
            dist_matrix[i, j] = dist
            dist_matrix[j, i] = dist
    return dist_matrix

def spxy_split(X, y, train_size=0.7, random_state=None):
    """
    SPXY algorithm để chia dữ liệu giữ tỉ lệ các lớp

    Parameters:
    -----------
    X : array-like, shape (n_samples, n_features)
    y : array-like, shape (n_samples,)
    train_size : float, tỷ lệ dữ liệu train
    random_state : int, seed cho random

    Returns:
    --------
    X_train, X_test, y_train, y_test
    """
    if random_state:
        np.random.seed(random_state)

    # Lấy các lớp duy nhất
    classes = np.unique(y)

    train_indices = []
    test_indices = []

    # Áp dụng SPXY cho từng lớp
    for cls in classes:
        cls_indices = np.where(y == cls)[0]
        X_cls = X[cls_indices]

        n_train = int(len(cls_indices) * train_size)

        if n_train < 2:
            # Nếu lớp quá nhỏ, chia ngẫu nhiên
            np.random.shuffle(cls_indices)
            train_indices.extend(cls_indices[:n_train])
            test_indices.extend(cls_indices[n_train:])
            continue

        # Tính ma trận khoảng cách
        dist_matrix = calculate_distance_matrix(X_cls)

        # Chọn 2 mẫu xa nhất
        max_dist_idx = np.unravel_index(dist_matrix.argmax(), dist_matrix.shape)
        selected = list(max_dist_idx)
        remaining = list(set(range(len(X_cls))) - set(selected))

        # Chọn các mẫu tiếp theo
        while len(selected) < n_train and remaining:
            max_min_dist = -1
            max_min_idx = -1

            for idx in remaining:
                min_dist = min([dist_matrix[idx, s] for s in selected])
                if min_dist > max_min_dist:
                    max_min_dist = min_dist
                    max_min_idx = idx

            if max_min_idx != -1:
                selected.append(max_min_idx)
                remaining.remove(max_min_idx)
            else:
                break

        # Nếu chưa đủ, thêm ngẫu nhiên
        if len(selected) < n_train:
            add_more = np.random.choice(remaining, n_train - len(selected), replace=False)
            selected.extend(add_more)
            remaining = list(set(remaining) - set(add_more))

        # Thêm vào danh sách train và test
        train_indices.extend(cls_indices[selected])
        test_indices.extend(cls_indices[remaining])

    # Trộn dữ liệu
    np.random.shuffle(train_indices)
    np.random.shuffle(test_indices)

    return X[train_indices], X[test_indices], y[train_indices], y[test_indices]

# ============================================================================
# DATA LOADING AND PREPROCESSING
# ============================================================================

def load_data():
    """Đọc dữ liệu từ Excel"""
    print("=" * 80)
    print("BẮT ĐẦU ĐỌC DỮ LIỆU")
    print("=" * 80)

    # Kiểm tra trường hợp nào
    if SINGLE_FILE_PATH:
        print(f"📁 Trường hợp 1: Đọc từ file đơn: {SINGLE_FILE_PATH}")
        df = pd.read_excel(SINGLE_FILE_PATH)

        # Kiểm tra cột Class
        if 'Class' not in df.columns:
            raise ValueError("File phải có cột 'Class'!")

        print(f"✓ Đọc thành công {len(df)} mẫu")
        print(f"✓ Các cột: {list(df.columns)}")
        print(f"✓ Phân bố lớp:\n{df['Class'].value_counts().sort_index()}")

        # Tách Class
        y = df['Class'].values

        # Lấy features và chuyển sang numeric
        print("\n🔄 Đang chuyển đổi dữ liệu sang numeric...")
        feature_df = df.drop('Class', axis=1).apply(pd.to_numeric, errors='coerce')
        feature_names = feature_df.columns.tolist()

        # Kiểm tra NaN
        nan_count = feature_df.isna().sum().sum()
        if nan_count > 0:
            print(f"⚠️  Phát hiện {nan_count} giá trị NaN - sẽ thay bằng median")
            for col in feature_df.columns:
                median_val = feature_df[col].median()
                if pd.isna(median_val):
                    median_val = 0
                feature_df[col].fillna(median_val, inplace=True)

        X = feature_df.values.astype(np.float64)
        print(f"✓ Chuyển đổi thành công: {X.shape[1]} features")

        print(f"\n🔄 Chia dữ liệu với SPXY (Train: {TRAIN_RATIO*100}%, Test: {(1-TRAIN_RATIO)*100}%)")
        X_train, X_test, y_train, y_test = spxy_split(X, y, train_size=TRAIN_RATIO, random_state=RANDOM_STATE)

        print(f"✓ Train: {len(X_train)} mẫu, Test: {len(X_test)} mẫu")

        # Lưu file train và test
        os.makedirs(OUTPUT_DIR, exist_ok=True)

        train_df = pd.DataFrame(X_train, columns=feature_names)
        train_df['Class'] = y_train
        train_df.to_excel(f"{OUTPUT_DIR}/train_data.xlsx", index=False)
        print(f"✓ Đã lưu: {OUTPUT_DIR}/train_data.xlsx")

        test_df = pd.DataFrame(X_test, columns=feature_names)
        test_df['Class'] = y_test
        test_df.to_excel(f"{OUTPUT_DIR}/test_data.xlsx", index=False)
        print(f"✓ Đã lưu: {OUTPUT_DIR}/test_data.xlsx")

    elif TRAIN_FILE_PATH and TEST_FILE_PATH:
        print(f"📁 Trường hợp 2: Đọc từ 2 files đã chia sẵn")
        print(f"   Train: {TRAIN_FILE_PATH}")
        print(f"   Test: {TEST_FILE_PATH}")

        train_df = pd.read_excel(TRAIN_FILE_PATH)
        test_df = pd.read_excel(TEST_FILE_PATH)

        # Kiểm tra cột Class
        if 'Class' not in train_df.columns or 'Class' not in test_df.columns:
            raise ValueError("Cả 2 file phải có cột 'Class'!")

        print(f"✓ Train: {len(train_df)} mẫu")
        print(f"✓ Test: {len(test_df)} mẫu")

        # Tách features và class
        y_train = train_df['Class'].values
        y_test = test_df['Class'].values

        # Lấy feature columns
        feature_cols = train_df.drop('Class', axis=1)
        feature_names = feature_cols.columns.tolist()

        # Chuyển đổi tất cả features sang numeric, thay thế non-numeric thành NaN
        print("\n🔄 Đang chuyển đổi dữ liệu sang numeric...")
        X_train_df = train_df.drop('Class', axis=1).apply(pd.to_numeric, errors='coerce')
        X_test_df = test_df.drop('Class', axis=1).apply(pd.to_numeric, errors='coerce')

        # Kiểm tra NaN values
        train_nans = X_train_df.isna().sum().sum()
        test_nans = X_test_df.isna().sum().sum()

        if train_nans > 0 or test_nans > 0:
            print(f"⚠️  Phát hiện {train_nans} NaN trong train và {test_nans} NaN trong test")
            print("⚠️  Các giá trị NaN sẽ được thay thế bằng median của feature đó")

            # Thay thế NaN bằng median
            for col in X_train_df.columns:
                median_val = X_train_df[col].median()
                if pd.isna(median_val):  # Nếu cả cột đều là NaN
                    median_val = 0
                X_train_df[col].fillna(median_val, inplace=True)
                X_test_df[col].fillna(median_val, inplace=True)

        # Convert to numpy arrays
        X_train = X_train_df.values.astype(np.float64)
        X_test = X_test_df.values.astype(np.float64)

        print(f"✓ Chuyển đổi thành công: {X_train.shape[1]} features")

    else:
        raise ValueError("Vui lòng cung cấp đường dẫn file!")

    return X_train, X_test, y_train, y_test, feature_names

def plot_data_distribution(X_train, X_test, y_train, y_test, feature_names):
    """Vẽ biểu đồ Bar chart Train vs Test (Mean ± Std) cho tất cả features"""
    print("\n" + "=" * 80)
    print("TẠO BIỂU ĐỒ PHÂN BỐ DỮ LIỆU")
    print("=" * 80)

    # Tạo thư mục output nếu chưa có
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    n_features = len(feature_names)

    # Tính mean và std cho train và test
    train_means = []
    train_stds = []
    test_means = []
    test_stds = []

    for i in range(n_features):
        train_means.append(X_train[:, i].mean())
        train_stds.append(X_train[:, i].std())
        test_means.append(X_test[:, i].mean())
        test_stds.append(X_test[:, i].std())

    # Tạo figure
    fig, ax = plt.subplots(figsize=(20, 6))

    # Vị trí các bars
    x = np.arange(n_features)
    width = 0.35

    # Vẽ bars với error bars
    bars1 = ax.bar(x - width/2, train_means, width, yerr=train_stds,
                   label='Train (Mean ± Std)', color='cornflowerblue',
                   capsize=3, alpha=0.8, edgecolor='black', linewidth=0.5)
    bars2 = ax.bar(x + width/2, test_means, width, yerr=test_stds,
                   label='Test (Mean ± Std)', color='coral',
                   capsize=3, alpha=0.8, edgecolor='black', linewidth=0.5)

    # Thiết lập labels và title
    ax.set_xlabel('Features', fontsize=12, fontweight='bold')
    ax.set_ylabel('Values', fontsize=12, fontweight='bold')
    ax.set_title('Data Distribution: Train vs Test (Mean ± Std)', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(feature_names, rotation=45, ha='right', fontsize=9)
    ax.legend(loc='upper right', fontsize=11)
    ax.grid(axis='y', alpha=0.3, linestyle='--')

    # Tight layout
    plt.tight_layout()
    plt.savefig(f"{OUTPUT_DIR}/data_distribution.png", dpi=300, bbox_inches='tight')
    print(f"✓ Đã lưu biểu đồ: {OUTPUT_DIR}/data_distribution.png")
    plt.close()

# ============================================================================
# MODEL TRAINING WITH OPTUNA
# ============================================================================

def get_model_configs():
    """Định nghĩa tất cả các mô hình ML"""
    return {
        # Support Vector Machines
        "SVM_linear": {"family": "SVM", "model_class": SVC, "kernel": "linear"},
        "SVM_rbf": {"family": "SVM", "model_class": SVC, "kernel": "rbf"},
        "SVM_poly": {"family": "SVM", "model_class": SVC, "kernel": "poly"},
        "SVM_sigmoid": {"family": "SVM", "model_class": SVC, "kernel": "sigmoid"},
        "NuSVC_linear": {"family": "SVM", "model_class": NuSVC, "kernel": "linear"},
        "NuSVC_rbf": {"family": "SVM", "model_class": NuSVC, "kernel": "rbf"},
        "NuSVC_poly": {"family": "SVM", "model_class": NuSVC, "kernel": "poly"},
        "NuSVC_sigmoid": {"family": "SVM", "model_class": NuSVC, "kernel": "sigmoid"},
        "LinearSVC": {"family": "SVM", "model_class": LinearSVC, "kernel": None},

        # Tree-based Models
        "DecisionTree": {"family": "Tree", "model_class": DecisionTreeClassifier},
        "RandomForest": {"family": "Tree", "model_class": RandomForestClassifier},
        "ExtraTrees": {"family": "Tree", "model_class": ExtraTreesClassifier},
        "GradientBoosting": {"family": "Tree", "model_class": GradientBoostingClassifier},
        "AdaBoost": {"family": "Tree", "model_class": AdaBoostClassifier},

        # Neural Networks
        "MLP_relu": {"family": "Neural Network", "model_class": MLPClassifier, "activation": "relu"},
        "MLP_tanh": {"family": "Neural Network", "model_class": MLPClassifier, "activation": "tanh"},
        "MLP_logistic": {"family": "Neural Network", "model_class": MLPClassifier, "activation": "logistic"},

        # K-Nearest Neighbors
        "KNN_uniform": {"family": "KNN", "model_class": KNeighborsClassifier, "weights": "uniform"},
        "KNN_distance": {"family": "KNN", "model_class": KNeighborsClassifier, "weights": "distance"},

        # Discriminant Analysis
        "LDA": {"family": "Discriminant", "model_class": LinearDiscriminantAnalysis},
        "QDA": {"family": "Discriminant", "model_class": QuadraticDiscriminantAnalysis},

        # Naive Bayes
        "GaussianNB": {"family": "Naive Bayes", "model_class": GaussianNB},
        "BernoulliNB": {"family": "Naive Bayes", "model_class": BernoulliNB},
    }

def objective_svc(trial, X_train, y_train, kernel):
    """Objective function cho SVC với pruning"""
    params = {
        'C': trial.suggest_float('C', 0.1, 100, log=True),
        'kernel': kernel,
        'random_state': RANDOM_STATE
    }

    if kernel == 'rbf' or kernel == 'sigmoid':
        params['gamma'] = trial.suggest_categorical('gamma', ['scale', 'auto'])
    elif kernel == 'poly':
        params['degree'] = trial.suggest_int('degree', 2, 5)
        params['gamma'] = trial.suggest_categorical('gamma', ['scale', 'auto'])
        params['coef0'] = trial.suggest_float('coef0', 0.0, 10.0)

    model = SVC(**params)
    cv = StratifiedKFold(n_splits=CV_FOLDS, shuffle=True, random_state=RANDOM_STATE)

    # Cross-validation với pruning
    scores = []
    for fold, (train_idx, val_idx) in enumerate(cv.split(X_train, y_train)):
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]

        model.fit(X_tr, y_tr)
        score = model.score(X_val, y_val)
        scores.append(score)

        # Report intermediate value cho pruning
        trial.report(score, fold)

        # Pruning: dừng sớm nếu kết quả kém
        if trial.should_prune():
            raise optuna.TrialPruned()

    return sum(scores) / len(scores)

def objective_nusvc(trial, X_train, y_train, kernel):
    """Objective function cho NuSVC với pruning"""
    params = {
        'nu': trial.suggest_float('nu', 0.01, 0.99),
        'kernel': kernel,
        'random_state': RANDOM_STATE
    }

    if kernel == 'rbf' or kernel == 'sigmoid':
        params['gamma'] = trial.suggest_categorical('gamma', ['scale', 'auto'])
    elif kernel == 'poly':
        params['degree'] = trial.suggest_int('degree', 2, 5)
        params['gamma'] = trial.suggest_categorical('gamma', ['scale', 'auto'])
        params['coef0'] = trial.suggest_float('coef0', 0.0, 10.0)

    model = NuSVC(**params)
    cv = StratifiedKFold(n_splits=CV_FOLDS, shuffle=True, random_state=RANDOM_STATE)

    scores = []
    for fold, (train_idx, val_idx) in enumerate(cv.split(X_train, y_train)):
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]

        model.fit(X_tr, y_tr)
        score = model.score(X_val, y_val)
        scores.append(score)

        trial.report(score, fold)
        if trial.should_prune():
            raise optuna.TrialPruned()

    return sum(scores) / len(scores)

def objective_tree(trial, X_train, y_train, model_class):
    """Objective function cho Tree-based models"""
    if model_class == DecisionTreeClassifier:
        params = {
            'max_depth': trial.suggest_int('max_depth', 3, 20),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
            'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
            'random_state': RANDOM_STATE
        }
    elif model_class in [RandomForestClassifier, ExtraTreesClassifier]:
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 300),
            'max_depth': trial.suggest_int('max_depth', 3, 20),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
            'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
            'random_state': RANDOM_STATE,
            'n_jobs': -1
        }
    elif model_class == GradientBoostingClassifier:
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 200),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
            'random_state': RANDOM_STATE
        }
    else:  # AdaBoost
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 200),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 2.0, log=True),
            'random_state': RANDOM_STATE
        }

    model = model_class(**params)
    cv = StratifiedKFold(n_splits=CV_FOLDS, shuffle=True, random_state=RANDOM_STATE)
    scores = cross_validate(model, X_train, y_train, cv=cv, scoring='accuracy', n_jobs=-1)

    return scores['test_score'].mean()

def objective_mlp(trial, X_train, y_train, activation):
    """Objective function cho MLP"""
    hidden_layer_sizes = []
    n_layers = trial.suggest_int('n_layers', 1, 3)
    for i in range(n_layers):
        hidden_layer_sizes.append(trial.suggest_int(f'n_units_l{i}', 10, 200))

    params = {
        'hidden_layer_sizes': tuple(hidden_layer_sizes),
        'activation': activation,
        'solver': trial.suggest_categorical('solver', ['adam', 'sgd']),
        'alpha': trial.suggest_float('alpha', 1e-5, 1e-1, log=True),
        'learning_rate': trial.suggest_categorical('learning_rate', ['constant', 'adaptive']),
        'max_iter': 1000,
        'random_state': RANDOM_STATE
    }

    model = MLPClassifier(**params)
    cv = StratifiedKFold(n_splits=CV_FOLDS, shuffle=True, random_state=RANDOM_STATE)
    scores = cross_validate(model, X_train, y_train, cv=cv, scoring='accuracy', n_jobs=-1)

    return scores['test_score'].mean()

def objective_knn(trial, X_train, y_train, weights):
    """Objective function cho KNN"""
    params = {
        'n_neighbors': trial.suggest_int('n_neighbors', 3, 30),
        'weights': weights,
        'metric': trial.suggest_categorical('metric', ['euclidean', 'manhattan', 'minkowski']),
        'n_jobs': -1
    }

    if params['metric'] == 'minkowski':
        params['p'] = trial.suggest_int('p', 1, 5)

    model = KNeighborsClassifier(**params)
    cv = StratifiedKFold(n_splits=CV_FOLDS, shuffle=True, random_state=RANDOM_STATE)
    scores = cross_validate(model, X_train, y_train, cv=cv, scoring='accuracy', n_jobs=-1)

    return scores['test_score'].mean()

def objective_lda(trial, X_train, y_train):
    """Objective function cho LDA"""
    params = {
        'solver': trial.suggest_categorical('solver', ['svd', 'lsqr', 'eigen']),
    }

    if params['solver'] in ['lsqr', 'eigen']:
        params['shrinkage'] = trial.suggest_float('shrinkage', 0.0, 1.0)

    model = LinearDiscriminantAnalysis(**params)
    cv = StratifiedKFold(n_splits=CV_FOLDS, shuffle=True, random_state=RANDOM_STATE)
    scores = cross_validate(model, X_train, y_train, cv=cv, scoring='accuracy', n_jobs=-1)

    return scores['test_score'].mean()

def objective_qda(trial, X_train, y_train):
    """Objective function cho QDA"""
    params = {
        'reg_param': trial.suggest_float('reg_param', 0.0, 1.0),
    }

    model = QuadraticDiscriminantAnalysis(**params)
    cv = StratifiedKFold(n_splits=CV_FOLDS, shuffle=True, random_state=RANDOM_STATE)
    scores = cross_validate(model, X_train, y_train, cv=cv, scoring='accuracy', n_jobs=-1)

    return scores['test_score'].mean()

def objective_naive_bayes(trial, X_train, y_train, model_class):
    """Objective function cho Naive Bayes"""
    if model_class == GaussianNB:
        params = {
            'var_smoothing': trial.suggest_float('var_smoothing', 1e-10, 1e-5, log=True)
        }
    elif model_class == BernoulliNB:
        params = {
            'alpha': trial.suggest_float('alpha', 0.1, 10.0, log=True),
            'binarize': trial.suggest_float('binarize', 0.0, 1.0)
        }

    model = model_class(**params)
    cv = StratifiedKFold(n_splits=CV_FOLDS, shuffle=True, random_state=RANDOM_STATE)
    scores = cross_validate(model, X_train, y_train, cv=cv, scoring='accuracy', n_jobs=-1)

    return scores['test_score'].mean()

def objective_linearsvc(trial, X_train, y_train):
    """Objective function cho LinearSVC với pruning"""
    params = {
        'C': trial.suggest_float('C', 0.1, 100, log=True),
        'loss': trial.suggest_categorical('loss', ['hinge', 'squared_hinge']),
        'max_iter': 10000,
        'random_state': RANDOM_STATE
    }

    model = LinearSVC(**params)
    cv = StratifiedKFold(n_splits=CV_FOLDS, shuffle=True, random_state=RANDOM_STATE)

    scores = []
    for fold, (train_idx, val_idx) in enumerate(cv.split(X_train, y_train)):
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]

        model.fit(X_tr, y_tr)
        score = model.score(X_val, y_val)
        scores.append(score)

        trial.report(score, fold)
        if trial.should_prune():
            raise optuna.TrialPruned()

    return sum(scores) / len(scores)

def train_single_model(model_name, config, X_train, y_train, X_test, y_test, scaler):
    """Train một model với Optuna"""
    print(f"\n{'='*60}")
    print(f"Training: {model_name}")
    print(f"{'='*60}")

    try:
        # Tạo study Optuna với Pruner để early stopping
        study = optuna.create_study(
            direction='maximize',
            sampler=TPESampler(seed=RANDOM_STATE),
            pruner=MedianPruner(
                n_startup_trials=MIN_TRIALS_BEFORE_PRUNING,
                n_warmup_steps=PRUNING_WARMUP_STEPS,
                interval_steps=1
            )
        )

        # Optimize dựa trên family
        model_class = config['model_class']

        if model_class in [SVC, NuSVC]:
            study.optimize(
                lambda trial: objective_svc(trial, X_train, y_train, config['kernel']) if model_class == SVC
                else objective_nusvc(trial, X_train, y_train, config['kernel']),
                n_trials=N_TRIALS,
                timeout=OPTUNA_TIMEOUT,
                show_progress_bar=False  # Tắt để giảm spam
            )
        elif model_class == LinearSVC:
            study.optimize(
                lambda trial: objective_linearsvc(trial, X_train, y_train),
                n_trials=N_TRIALS,
                timeout=OPTUNA_TIMEOUT,
                show_progress_bar=False
            )
        elif config['family'] == 'Tree':
            study.optimize(
                lambda trial: objective_tree(trial, X_train, y_train, model_class),
                n_trials=N_TRIALS,
                timeout=OPTUNA_TIMEOUT,
                show_progress_bar=False
            )
        elif model_class == MLPClassifier:
            study.optimize(
                lambda trial: objective_mlp(trial, X_train, y_train, config['activation']),
                n_trials=N_TRIALS,
                timeout=OPTUNA_TIMEOUT,
                show_progress_bar=False
            )
        elif model_class == KNeighborsClassifier:
            study.optimize(
                lambda trial: objective_knn(trial, X_train, y_train, config['weights']),
                n_trials=N_TRIALS,
                timeout=OPTUNA_TIMEOUT,
                show_progress_bar=False
            )
        elif model_class == LinearDiscriminantAnalysis:
            study.optimize(
                lambda trial: objective_lda(trial, X_train, y_train),
                n_trials=N_TRIALS,
                timeout=OPTUNA_TIMEOUT,
                show_progress_bar=False
            )
        elif model_class == QuadraticDiscriminantAnalysis:
            study.optimize(
                lambda trial: objective_qda(trial, X_train, y_train),
                n_trials=N_TRIALS,
                timeout=OPTUNA_TIMEOUT,
                show_progress_bar=False
            )
        elif model_class in [GaussianNB, BernoulliNB]:
            study.optimize(
                lambda trial: objective_naive_bayes(trial, X_train, y_train, model_class),
                n_trials=N_TRIALS,
                timeout=OPTUNA_TIMEOUT,
                show_progress_bar=False
            )

        # Lấy best params
        best_params = study.best_params

        # Thống kê trials
        n_complete = len([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE])
        n_pruned = len([t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED])
        n_failed = len([t for t in study.trials if t.state == optuna.trial.TrialState.FAIL])

        print(f"✓ Trials: {n_complete} hoàn thành, {n_pruned} bị prune (dừng sớm), {n_failed} thất bại")
        print(f"✓ Best params: {best_params}")
        print(f"✓ Best CV score: {study.best_value:.4f}")

        # Train model với best params
        if 'kernel' in config and config['kernel']:
            best_params['kernel'] = config['kernel']
        if 'activation' in config:
            best_params['activation'] = config['activation']
        if 'weights' in config:
            best_params['weights'] = config['weights']
        if model_class == LinearSVC:
            best_params['max_iter'] = 10000
        if model_class == MLPClassifier:
            best_params['max_iter'] = 1000

        # Loại bỏ các params không hợp lệ
        params_to_remove = []

        # MLP: n_layers và n_units_l* là params trung gian, không phải params của MLPClassifier
        if model_class == MLPClassifier:
            params_to_remove.extend([k for k in best_params.keys() if k.startswith('n_units_l') or k == 'n_layers'])

        # KNN không có random_state
        if model_class == KNeighborsClassifier:
            params_to_remove.append('random_state')

        # Naive Bayes không có random_state
        if model_class in [GaussianNB, BernoulliNB]:
            params_to_remove.append('random_state')

        # LDA/QDA không có random_state
        if model_class in [LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis]:
            params_to_remove.append('random_state')

        # Xóa các params không hợp lệ
        for param in params_to_remove:
            best_params.pop(param, None)

        # Thêm random_state cho các model support
        if model_class not in [KNeighborsClassifier, GaussianNB, BernoulliNB,
                                LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis]:
            best_params['random_state'] = RANDOM_STATE

        model = model_class(**best_params)
        model.fit(X_train, y_train)

        # Cross-validation metrics
        cv = StratifiedKFold(n_splits=CV_FOLDS, shuffle=True, random_state=RANDOM_STATE)
        cv_results = cross_validate(
            model, X_train, y_train, cv=cv,
            scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'],
            n_jobs=-1
        )

        # Test metrics
        y_pred = model.predict(X_test)

        # Prediction speed
        start_time = time.time()
        _ = model.predict(X_test)
        elapsed_time = time.time() - start_time
        pred_speed = len(X_test) / elapsed_time if elapsed_time > 0 else 0

        # Model size
        model_size = len(pickle.dumps(model))

        # Confusion matrix
        cm = confusion_matrix(y_test, y_pred)

        results = {
            'Algorithm Family': config.get('family', 'SVM'),
            'Model': model_name,
            'Kernel/Type': config.get('kernel') or config.get('activation') or config.get('weights') or 'N/A',
            'Best params (JSON)': json.dumps(best_params),
            'CV Train Accuracy': cv_results['test_accuracy'].mean(),
            'CV Train Precision': cv_results['test_precision_macro'].mean(),
            'CV Train Recall': cv_results['test_recall_macro'].mean(),
            'CV Train F1': cv_results['test_f1_macro'].mean(),
            'Test Accuracy': accuracy_score(y_test, y_pred),
            'Test Precision': precision_score(y_test, y_pred, average='macro'),
            'Test Recall': recall_score(y_test, y_pred, average='macro'),
            'Test F1': f1_score(y_test, y_pred, average='macro'),
            'Prediction speed (obs/sec)': pred_speed,
            'Model size (bytes)': model_size
        }

        print(f"✓ Test Accuracy: {results['Test Accuracy']:.4f}")
        print(f"✓ Test Precision: {results['Test Precision']:.4f}")
        print(f"✓ Test F1: {results['Test F1']:.4f}")

        return model, results, cm

    except Exception as e:
        print(f"\n❌ LỖI khi train {model_name}!")
        print(f"   Loại lỗi: {type(e).__name__}")
        print(f"   Chi tiết: {str(e)[:200]}")
        import traceback
        print(f"   Traceback (rút gọn):")
        tb_lines = traceback.format_exc().split('\n')
        for line in tb_lines[-5:]:  # Chỉ hiển 5 dòng cuối
            if line.strip():
                print(f"     {line}")

        # Trả về None để bỏ qua model này
        return None, None, None

def train_all_models(X_train, y_train, X_test, y_test):
    """Train tất cả các models"""
    print("\n" + "=" * 80)
    print("BẮT ĐẦU TRAINING TẤT CẢ CÁC MODELS")
    print("=" * 80)

    # Tắt optuna logging để giảm spam
    optuna.logging.set_verbosity(optuna.logging.WARNING)

    # Chuẩn hóa dữ liệu
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model_configs = get_model_configs()

    all_results = []
    trained_models = {}
    confusion_matrices = {}

    for model_name, config in model_configs.items():
        try:
            model, results, cm = train_single_model(
                model_name, config, X_train_scaled, y_train, X_test_scaled, y_test, scaler
            )

            # Chỉ thêm vào kết quả nếu train thành công
            if model is not None and results is not None:
                all_results.append(results)
                trained_models[model_name] = {'model': model, 'scaler': scaler, 'family': config.get('family', 'SVM')}
                confusion_matrices[model_name] = cm
            else:
                print(f"⚠️  Bỏ qua {model_name} do train thất bại\n")

        except Exception as e:
            print(f"❌ Lỗi ngoại lệ khi train {model_name}: {e}\n")
            continue

    return all_results, trained_models, confusion_matrices

def save_results(all_results, confusion_matrices):
    """Lưu kết quả training vào Excel và plot confusion matrices"""
    print("\n" + "=" * 80)
    print("LƯƯ KÉT QUẢ")
    print("=" * 80)

    # Tạo thư mục output
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # Lưu kết quả vào Excel
    results_df = pd.DataFrame(all_results)
    # Kết quả sẽ được gắc vào results.xlsx khi dùng Gradio

    # Hiển thị top 10 models theo Test Precision
    print("\nTop 10 models theo Test Precision:")
    top_models = results_df.nlargest(10, 'Test Precision')[['Model', 'Test Precision', 'Test Accuracy', 'Test F1']]
    print(top_models.to_string(index=False))

    # Vẽ confusion matrices
    cm_dir = f"{OUTPUT_DIR}/confusion_matrices"
    os.makedirs(cm_dir, exist_ok=True)

    for model_name, cm in confusion_matrices.items():
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.title(f'Confusion Matrix - {model_name}')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.tight_layout()

        safe_name = model_name.replace('-', '_').replace(' ', '_')
        plt.savefig(f"{cm_dir}/{safe_name}_confusion_matrix.png", dpi=100, bbox_inches='tight')
        plt.close()

    print(f"✓ Đã lưu {len(confusion_matrices)} confusion matrices tại: {cm_dir}/")

def select_models_for_arduino(all_results, trained_models):
    """Cho phép người dùng chọn models để tạo Arduino library"""
    print("\n" + "=" * 80)
    print("CHỌN MODELS ĐỂ TẠO ARDUINO LIBRARY")
    print("=" * 80)

    # Sắp xếp theo Test Precision
    results_df = pd.DataFrame(all_results)
    results_df = results_df.sort_values('Test Precision', ascending=False).reset_index(drop=True)

    # Hiển thị bảng với format đẹp hơn
    print("\n📊 DANH SÁCH TẤT CẢ CÁC MODELS (sắp xếp theo Test Precision):")
    print("=" * 100)
    print(f"{'STT':<5} {'Model':<30} {'Family':<18} {'Precision':<12} {'Accuracy':<12} {'F1-Score':<12}")
    print("-" * 100)

    for idx, row in results_df.iterrows():
        print(f"{idx+1:<5} {row['Model']:<30} {row['Algorithm Family']:<18} "
              f"{row['Test Precision']:<12.4f} {row['Test Accuracy']:<12.4f} {row['Test F1']:<12.4f}")

    print("=" * 100)

    # Chọn models
    selected_models = {}

    if IN_COLAB:
        # GOOGLE COLAB: Sử dụng input() - sẽ hiện input box ngay dưới cell
        print("\n📌 HƯỚNG DẪN CHỌN MODELS:")
        print("   • Nhấn Enter (để trống): Tự động chọn 10 models tốt nhất")
        print("   • Nhập các số: Chọn models theo STT (có thể chọn bao nhiêu tùy ý)")
        print("     Ví dụ: '1 2 3 4 5' hoặc '1,2,3,4,5' (chọn 5 models đầu)")
        print("     Ví dụ: '1 5 10 15' (chọn 4 models cụ thể)")
        print()

        user_input = input("👉 Nhập STT các models (cách nhau bởi dấu cách hoặc dấu phẩy): ").strip()

        # Xử lý input
        if not user_input:
            # AUTO MODE - không nhập gì
            top_n = min(10, len(results_df))
            print(f"\n✅ Chế độ AUTO: Đã chọn {top_n} models có Test Precision cao nhất")
            print("-" * 80)

            for i in range(top_n):
                model_name = results_df.iloc[i]['Model']
                selected_models[model_name] = trained_models[model_name]
                print(f"   {i+1:2d}. {model_name:<30} → Precision: {results_df.iloc[i]['Test Precision']:.4f}")
        else:
            # MANUAL MODE - nhập các số
            try:
                # Parse numbers - hỗ trợ cả dấu cách và dấu phẩy
                user_input = user_input.replace(',', ' ')  # Chuyển dấu phẩy thành dấu cách
                numbers = []

                for x in user_input.split():
                    x = x.strip()
                    if x.isdigit():
                        num = int(x)
                        if 1 <= num <= len(results_df):
                            if num not in numbers:  # Tránh trùng lặp
                                numbers.append(num)
                        else:
                            print(f"⚠️  Bỏ qua số {num} (ngoài phạm vi 1-{len(results_df)})")

                if numbers:
                    print(f"\n✅ Đã chọn {len(numbers)} models:")
                    print("-" * 80)

                    for num in sorted(numbers):  # Sắp xếp để hiển thị đẹp
                        model_name = results_df.iloc[num - 1]['Model']
                        selected_models[model_name] = trained_models[model_name]
                        print(f"   {num:2d}. {model_name:<30} → Precision: {results_df.iloc[num - 1]['Test Precision']:.4f}")
                else:
                    # Không có số hợp lệ -> fallback AUTO
                    print(f"\n⚠️  Không có số hợp lệ nào! Sử dụng AUTO mode (10 models tốt nhất)...")
                    print("-" * 80)
                    top_n = min(10, len(results_df))
                    for i in range(top_n):
                        model_name = results_df.iloc[i]['Model']
                        selected_models[model_name] = trained_models[model_name]
                        print(f"   {i+1:2d}. {model_name:<30} → Precision: {results_df.iloc[i]['Test Precision']:.4f}")

            except Exception as e:
                print(f"\n⚠️  Lỗi khi xử lý input: {e}")
                print("    Sử dụng AUTO mode (10 models tốt nhất)...")
                print("-" * 80)
                top_n = min(10, len(results_df))
                for i in range(top_n):
                    model_name = results_df.iloc[i]['Model']
                    selected_models[model_name] = trained_models[model_name]
                    print(f"   {i+1:2d}. {model_name:<30} → Precision: {results_df.iloc[i]['Test Precision']:.4f}")

    else:
        # LOCAL MODE: Tự động chọn AUTO
        print("\n🖥️  [Local mode] Tự động sử dụng AUTO mode - chọn 10 models tốt nhất")
        top_n = min(10, len(results_df))

        for i in range(top_n):
            model_name = results_df.iloc[i]['Model']
            selected_models[model_name] = trained_models[model_name]
            print(f"   {i+1:2d}. {model_name:<30} → Precision: {results_df.iloc[i]['Test Precision']:.4f}")

    # Tổng kết
    print("\n" + "=" * 100)
    print(f"✅ HOÀN TẤT: Đã chọn {len(selected_models)} models để tạo Arduino library")
    print("=" * 100 + "\n")

    return selected_models

def generate_svm_c_code(model, model_name, scaler, n_features, n_classes):
    """
    Generate C code cho SVM models (linear, poly, rbf, sigmoid)
    Thay thế m2cgen với custom lightweight implementation
    """
    safe_name = model_name.replace('-', '_').replace(' ', '_')

    # Kiểm tra kernel type
    if hasattr(model, 'kernel'):
        kernel = model.kernel
    else:
        # LinearSVC không có kernel attribute
        kernel = 'linear'

    # Lấy gamma
    if kernel != 'linear' and hasattr(model, 'gamma'):
        gamma = model.gamma if isinstance(model.gamma, float) else model._gamma
        if gamma == 'scale':
            gamma = 1.0 / (n_features * model.support_vectors_.var())
        elif gamma == 'auto':
            gamma = 1.0 / n_features
    else:
        gamma = 1.0

    coef0 = model.coef0 if hasattr(model, 'coef0') else 0.0
    degree = model.degree if hasattr(model, 'degree') else 3

    code_parts = []

    # Scaler parameters
    code_parts.append(f"""
// ========================================
// Model: {model_name} (Kernel: {kernel})
// ========================================

// Scaler parameters
static const double scaler_mean_{safe_name}[{n_features}] = {{
    {', '.join(f'{x:.10f}' for x in scaler.mean_)}
}};

static const double scaler_scale_{safe_name}[{n_features}] = {{
    {', '.join(f'{x:.10f}' for x in scaler.scale_)}
}};

// Standardize features
static inline void standardize_{safe_name}(const float* features, double* scaled) {{
    for (int i = 0; i < {n_features}; i++) {{
        scaled[i] = ((double)features[i] - scaler_mean_{safe_name}[i]) / scaler_scale_{safe_name}[i];
    }}
}}
""")

    # Kernel functions
    if kernel == 'poly':
        code_parts.append(f"""
// Polynomial kernel
static inline double poly_kernel_{safe_name}(const double* x, const double* sv) {{
    double dot = 0.0;
    for (int i = 0; i < {n_features}; i++) dot += x[i] * sv[i];
    double result = {gamma} * dot + {coef0};
    double powered = result;
    for (int d = 1; d < {degree}; d++) powered *= result;
    return powered;
}}
""")
    elif kernel == 'rbf':
        code_parts.append(f"""
// RBF kernel
static inline double rbf_kernel_{safe_name}(const double* x, const double* sv) {{
    double dist_sq = 0.0;
    for (int i = 0; i < {n_features}; i++) {{
        double diff = x[i] - sv[i];
        dist_sq += diff * diff;
    }}
    return exp(-{gamma} * dist_sq);
}}
""")
    elif kernel == 'sigmoid':
        code_parts.append(f"""
// Sigmoid kernel
static inline double sigmoid_kernel_{safe_name}(const double* x, const double* sv) {{
    double dot = 0.0;
    for (int i = 0; i < {n_features}; i++) dot += x[i] * sv[i];
    return tanh({gamma} * dot + {coef0});
}}
""")
    elif kernel == 'linear':
        code_parts.append(f"""
// Linear kernel (dot product)
static inline double linear_kernel_{safe_name}(const double* x, const double* sv) {{
    double dot = 0.0;
    for (int i = 0; i < {n_features}; i++) dot += x[i] * sv[i];
    return dot;
}}
""")

    # Support vectors và prediction
    if hasattr(model, 'support_vectors_'):
        # SVC/NuSVC
        sv = model.support_vectors_
        sv_lines = ',\n'.join('    {' + ', '.join(f'{x:.10f}' for x in vec) + '}' for vec in sv)

        if n_classes == 2:
            # Binary
            alpha = model.dual_coef_[0]
            intercept = model.intercept_[0]

            code_parts.append(f"""
static const double sv_{safe_name}[{len(sv)}][{n_features}] = {{
{sv_lines}
}};

static const double alpha_{safe_name}[{len(sv)}] = {{
    {', '.join(f'{a:.10f}' for a in alpha)}
}};

int predict_{safe_name}(float features[{n_features}]) {{
    double scaled[{n_features}];
    standardize_{safe_name}(features, scaled);

    double decision = {intercept:.10f};
    for (int i = 0; i < {len(sv)}; i++) {{
        decision += alpha_{safe_name}[i] * {kernel}_kernel_{safe_name}(scaled, sv_{safe_name}[i]);
    }}

    return decision >= 0 ? {model.classes_[1]} : {model.classes_[0]};
}}
""")
        else:
            # Multi-class OvO
            n_sv = model.n_support_
            dual_coef_lines = ',\n'.join('    {' + ', '.join(f'{x:.10f}' for x in row) + '}' for row in model.dual_coef_)

            code_parts.append(f"""
static const double sv_{safe_name}[{len(sv)}][{n_features}] = {{
{sv_lines}
}};

static const int n_support_{safe_name}[{n_classes}] = {{{', '.join(map(str, n_sv))}}};

static const double dual_coef_{safe_name}[{n_classes - 1}][{len(sv)}] = {{
{dual_coef_lines}
}};

static const double intercepts_{safe_name}[{len(model.intercept_)}] = {{
    {', '.join(f'{x:.10f}' for x in model.intercept_)}
}};

int predict_{safe_name}(float features[{n_features}]) {{
    double scaled[{n_features}];
    standardize_{safe_name}(features, scaled);

    int votes[{n_classes}] = {{0}};
    int clf_idx = 0;

    int sv_start[{n_classes + 1}];
    sv_start[0] = 0;
    for (int i = 0; i < {n_classes}; i++) {{
        sv_start[i + 1] = sv_start[i] + n_support_{safe_name}[i];
    }}

    for (int i = 0; i < {n_classes}; i++) {{
        for (int j = i + 1; j < {n_classes}; j++) {{
            double decision = intercepts_{safe_name}[clf_idx];

            for (int k = sv_start[i]; k < sv_start[i + 1]; k++) {{
                decision += dual_coef_{safe_name}[j - 1][k] * {kernel}_kernel_{safe_name}(scaled, sv_{safe_name}[k]);
            }}

            for (int k = sv_start[j]; k < sv_start[j + 1]; k++) {{
                decision += dual_coef_{safe_name}[i][k] * {kernel}_kernel_{safe_name}(scaled, sv_{safe_name}[k]);
            }}

            if (decision > 0) votes[i]++; else votes[j]++;
            clf_idx++;
        }}
    }}

    int max_votes = votes[0], predicted = 0;
    for (int i = 1; i < {n_classes}; i++) {{
        if (votes[i] > max_votes) {{
            max_votes = votes[i];
            predicted = i;
        }}
    }}

    static const int class_labels[{n_classes}] = {{{', '.join(map(str, model.classes_))}}};
    return class_labels[predicted];
}}
""")
    else:
        # LinearSVC - sử dụng coef_ và intercept_
        coef = model.coef_
        intercept = model.intercept_

        if n_classes == 2:
            code_parts.append(f"""
static const double coef_{safe_name}[{n_features}] = {{
    {', '.join(f'{x:.10f}' for x in coef[0])}
}};

int predict_{safe_name}(float features[{n_features}]) {{
    double scaled[{n_features}];
    standardize_{safe_name}(features, scaled);

    double decision = {intercept[0]:.10f};
    for (int i = 0; i < {n_features}; i++) {{
        decision += coef_{safe_name}[i] * scaled[i];
    }}

    return decision >= 0 ? {model.classes_[1]} : {model.classes_[0]};
}}
""")
        else:
            coef_lines = ',\n'.join('    {' + ', '.join(f'{x:.10f}' for x in row) + '}' for row in coef)

            code_parts.append(f"""
static const double coef_{safe_name}[{n_classes}][{n_features}] = {{
{coef_lines}
}};

static const double intercepts_{safe_name}[{n_classes}] = {{
    {', '.join(f'{x:.10f}' for x in intercept)}
}};

int predict_{safe_name}(float features[{n_features}]) {{
    double scaled[{n_features}];
    standardize_{safe_name}(features, scaled);

    double scores[{n_classes}];
    for (int c = 0; c < {n_classes}; c++) {{
        scores[c] = intercepts_{safe_name}[c];
        for (int i = 0; i < {n_features}; i++) {{
            scores[c] += coef_{safe_name}[c][i] * scaled[i];
        }}
    }}

    int predicted = 0;
    double max_score = scores[0];
    for (int c = 1; c < {n_classes}; c++) {{
        if (scores[c] > max_score) {{
            max_score = scores[c];
            predicted = c;
        }}
    }}

    static const int class_labels[{n_classes}] = {{{', '.join(map(str, model.classes_))}}};
    return class_labels[predicted];
}}
""")

    return '\n'.join(code_parts)

def generate_tree_c_code(model, model_name, scaler, n_features, n_classes):
    """Tạo code C cho Decision Tree-based models"""
    safe_name = model_name.replace('-', '_').replace(' ', '_')

    code_parts = []
    code_parts.append(f"""
// ========================================
// {model_name}
// ========================================

""")

    # Thêm scaler parameters
    code_parts.append(f"static const float {safe_name}_scaler_mean[{n_features}] = {{\n")
    for i, val in enumerate(scaler.mean_):
        code_parts.append(f"    {val:.10f}f{',' if i < len(scaler.mean_)-1 else ''}\n")
    code_parts.append("};\n\n")

    code_parts.append(f"static const float {safe_name}_scaler_scale[{n_features}] = {{\n")
    for i, val in enumerate(scaler.scale_):
        code_parts.append(f"    {val:.10f}f{',' if i < len(scaler.scale_)-1 else ''}\n")
    code_parts.append("};\n\n")

    # Tree-based models: extract decision tree structure
    # For single tree (DecisionTreeClassifier)
    if hasattr(model, 'tree_'):
        tree = model.tree_
        code_parts.append(f"""
static int {safe_name}_predict_tree(const float* features_scaled) {{
    // Simple tree traversal using arrays
    int node = 0;
    while (tree_feature[node] != -2) {{ // -2 indicates leaf
        if (features_scaled[tree_feature[node]] <= tree_threshold[node]) {{
            node = tree_children_left[node];
        }} else {{
            node = tree_children_right[node];
        }}
    }}
    // Return class from leaf node
    return tree_value[node];
}}
""")

    # For ensemble models (RandomForest, ExtraTrees, etc.)
    elif hasattr(model, 'estimators_'):
        n_estimators = len(model.estimators_)
        code_parts.append(f"""
// Ensemble model with {n_estimators} trees
// Note: Full implementation would be very large for ESP32
// Using simplified voting approach based on feature importance

static const float {safe_name}_feature_importance[{n_features}] = {{\n""")
        if hasattr(model, 'feature_importances_'):
            for i, val in enumerate(model.feature_importances_):
                code_parts.append(f"    {val:.10f}f{',' if i < n_features-1 else ''}\n")
        else:
            for i in range(n_features):
                code_parts.append(f"    {1.0/n_features:.10f}f{',' if i < n_features-1 else ''}\n")
        code_parts.append("};\n\n")

        code_parts.append(f"""
static int {safe_name}_predict_ensemble(const float* features_scaled) {{
    // Simplified ensemble: weighted vote based on feature importance
    float scores[{n_classes}] = {{0.0f}};

    for (int c = 0; c < {n_classes}; c++) {{
        for (int f = 0; f < {n_features}; f++) {{
            scores[c] += features_scaled[f] * {safe_name}_feature_importance[f];
        }}
    }}

    // Find max score
    int predicted = 0;
    float max_score = scores[0];
    for (int c = 1; c < {n_classes}; c++) {{
        if (scores[c] > max_score) {{
            max_score = scores[c];
            predicted = c;
        }}
    }}

    return predicted;
}}
""")

    # Main predict function
    code_parts.append(f"""
int predict_{safe_name}(float features[{n_features}]) {{
    // Apply scaling
    float features_scaled[{n_features}];
    for (int i = 0; i < {n_features}; i++) {{
        features_scaled[i] = (features[i] - {safe_name}_scaler_mean[i]) / {safe_name}_scaler_scale[i];
    }}

    int predicted;
""")

    if hasattr(model, 'tree_'):
        code_parts.append(f"    predicted = {safe_name}_predict_tree(features_scaled);\n")
    else:
        code_parts.append(f"    predicted = {safe_name}_predict_ensemble(features_scaled);\n")

    code_parts.append(f"""
    static const int class_labels[{n_classes}] = {{{', '.join(map(str, model.classes_))}}};
    return class_labels[predicted];
}}
""")

    return ''.join(code_parts)

def generate_mlp_c_code(model, model_name, scaler, n_features, n_classes):
    """Tạo code C cho Multi-Layer Perceptron"""
    safe_name = model_name.replace('-', '_').replace(' ', '_')

    code_parts = []
    code_parts.append(f"""
// ========================================
// {model_name}
// ========================================

""")

    # Scaler parameters
    code_parts.append(f"static const float {safe_name}_scaler_mean[{n_features}] = {{\n")
    for i, val in enumerate(scaler.mean_):
        code_parts.append(f"    {val:.10f}f{',' if i < len(scaler.mean_)-1 else ''}\n")
    code_parts.append("};\n\n")

    code_parts.append(f"static const float {safe_name}_scaler_scale[{n_features}] = {{\n")
    for i, val in enumerate(scaler.scale_):
        code_parts.append(f"    {val:.10f}f{',' if i < len(scaler.scale_)-1 else ''}\n")
    code_parts.append("};\n\n")

    # Activation function
    activation = getattr(model, 'activation', 'relu')  # Default to relu if not found
    code_parts.append(f"""
static inline float {safe_name}_activation(float x) {{""")

    if activation == 'relu':
        code_parts.append("""
    return (x > 0.0f) ? x : 0.0f;
}
""")
    elif activation == 'tanh':
        code_parts.append("""
    return tanhf(x);
}
""")
    elif activation == 'logistic':
        code_parts.append("""
    return 1.0f / (1.0f + expf(-x));
}
""")
    else:  # identity
        code_parts.append("""
    return x;
}
""")

    # Network weights and biases
    n_layers = len(model.coefs_)

    for layer_idx in range(n_layers):
        coef = model.coefs_[layer_idx]
        intercept = model.intercepts_[layer_idx]

        n_inputs = coef.shape[0]
        n_outputs = coef.shape[1]

        # Weight matrix
        code_parts.append(f"\nstatic const float {safe_name}_layer{layer_idx}_weights[{n_inputs}][{n_outputs}] = {{\n")
        for i in range(n_inputs):
            code_parts.append("    {")
            for j in range(n_outputs):
                code_parts.append(f"{coef[i][j]:.10f}f{',' if j < n_outputs-1 else ''}")
            code_parts.append(f"}}{',' if i < n_inputs-1 else ''}\n")
        code_parts.append("};\n")

        # Bias vector
        code_parts.append(f"\nstatic const float {safe_name}_layer{layer_idx}_bias[{n_outputs}] = {{\n")
        for i, val in enumerate(intercept):
            code_parts.append(f"    {val:.10f}f{',' if i < len(intercept)-1 else ''}\n")
        code_parts.append("};\n")

    # Predict function
    # Calculate max hidden layer size
    max_hidden_size = max([coef.shape[1] for coef in model.coefs_[:-1]] + [n_features])

    code_parts.append(f"""
int predict_{safe_name}(float features[{n_features}]) {{
    // Apply scaling
    float features_scaled[{n_features}];
    for (int i = 0; i < {n_features}; i++) {{
        features_scaled[i] = (features[i] - {safe_name}_scaler_mean[i]) / {safe_name}_scaler_scale[i];
    }}

    // Forward pass through network
    float layer_input[{max_hidden_size}];
    float layer_output[{max_hidden_size}];

    // Copy input
    for (int i = 0; i < {n_features}; i++) {{
        layer_input[i] = features_scaled[i];
    }}

""")

    # Generate forward pass for each layer
    for layer_idx in range(n_layers):
        n_inputs = model.coefs_[layer_idx].shape[0]
        n_outputs = model.coefs_[layer_idx].shape[1]
        is_output_layer = (layer_idx == n_layers - 1)

        code_parts.append(f"""    // Layer {layer_idx}
    for (int j = 0; j < {n_outputs}; j++) {{
        layer_output[j] = {safe_name}_layer{layer_idx}_bias[j];
        for (int i = 0; i < {n_inputs}; i++) {{
            layer_output[j] += layer_input[i] * {safe_name}_layer{layer_idx}_weights[i][j];
        }}
""")

        if not is_output_layer:
            code_parts.append(f"        layer_output[j] = {safe_name}_activation(layer_output[j]);\n")

        code_parts.append("    }\n")

        if not is_output_layer:
            code_parts.append(f"""
    // Copy to next layer input
    for (int i = 0; i < {n_outputs}; i++) {{
        layer_input[i] = layer_output[i];
    }}

""")

    code_parts.append(f"""
    // Find class with max output
    int predicted = 0;
    float max_output = layer_output[0];
    for (int i = 1; i < {n_classes}; i++) {{
        if (layer_output[i] > max_output) {{
            max_output = layer_output[i];
            predicted = i;
        }}
    }}

    static const int class_labels[{n_classes}] = {{{', '.join(map(str, model.classes_))}}};
    return class_labels[predicted];
}}
""")

    return ''.join(code_parts)

def generate_knn_c_code(model, model_name, scaler, n_features, n_classes):
    """Tạo code C cho K-Nearest Neighbors"""
    safe_name = model_name.replace('-', '_').replace(' ', '_')

    # KNN requires storing training data - can be very large
    # We'll store a subset or use a simplified approach
    n_training = min(len(model._fit_X), 100)  # Limit to 100 samples for ESP32

    code_parts = []
    code_parts.append(f"""
// ========================================
// {model_name}
// ========================================
// Note: Using first {n_training} training samples due to memory constraints

""")

    # Scaler
    code_parts.append(f"static const float {safe_name}_scaler_mean[{n_features}] = {{\n")
    for i, val in enumerate(scaler.mean_):
        code_parts.append(f"    {val:.10f}f{',' if i < len(scaler.mean_)-1 else ''}\n")
    code_parts.append("};\n\n")

    code_parts.append(f"static const float {safe_name}_scaler_scale[{n_features}] = {{\n")
    for i, val in enumerate(scaler.scale_):
        code_parts.append(f"    {val:.10f}f{',' if i < len(scaler.scale_)-1 else ''}\n")
    code_parts.append("};\n\n")

    # Training data
    code_parts.append(f"static const float {safe_name}_training_data[{n_training}][{n_features}] = {{\n")
    for i in range(n_training):
        code_parts.append("    {")
        for j in range(n_features):
            code_parts.append(f"{model._fit_X[i][j]:.10f}f{',' if j < n_features-1 else ''}")
        code_parts.append(f"}}{',' if i < n_training-1 else ''}\n")
    code_parts.append("};\n\n")

    code_parts.append(f"static const int {safe_name}_training_labels[{n_training}] = {{\n")
    for i in range(n_training):
        code_parts.append(f"    {model._y[i]}{',' if i < n_training-1 else ''}\n")
    code_parts.append("};\n\n")

    k = model.n_neighbors
    metric = getattr(model, 'metric', 'euclidean')  # Default to euclidean

    # Distance function
    code_parts.append(f"""
static inline float {safe_name}_distance(const float* a, const float* b) {{
    float dist = 0.0f;
""")

    if metric == 'euclidean':
        code_parts.append(f"""    for (int i = 0; i < {n_features}; i++) {{
        float diff = a[i] - b[i];
        dist += diff * diff;
    }}
    return sqrtf(dist);
}}
""")
    elif metric == 'manhattan':
        code_parts.append(f"""    for (int i = 0; i < {n_features}; i++) {{
        dist += fabsf(a[i] - b[i]);
    }}
    return dist;
}}
""")
    else:  # minkowski or other
        p = getattr(model, 'p', 2)
        code_parts.append(f"""    for (int i = 0; i < {n_features}; i++) {{
        dist += powf(fabsf(a[i] - b[i]), {p}.0f);
    }}
    return powf(dist, 1.0f/{p}.0f);
}}
""")

    # Predict function
    code_parts.append(f"""
int predict_{safe_name}(float features[{n_features}]) {{
    // Apply scaling
    float features_scaled[{n_features}];
    for (int i = 0; i < {n_features}; i++) {{
        features_scaled[i] = (features[i] - {safe_name}_scaler_mean[i]) / {safe_name}_scaler_scale[i];
    }}

    // Calculate distances to all training samples
    float distances[{n_training}];
    for (int i = 0; i < {n_training}; i++) {{
        distances[i] = {safe_name}_distance(features_scaled, {safe_name}_training_data[i]);
    }}

    // Find k nearest neighbors using simple selection
    int k_indices[{k}];
    float k_distances[{k}];

    // Initialize with first k samples
    for (int i = 0; i < {k}; i++) {{
        k_indices[i] = i;
        k_distances[i] = distances[i];
    }}

    // Find k smallest distances
    for (int i = {k}; i < {n_training}; i++) {{
        // Find max in current k
        int max_idx = 0;
        for (int j = 1; j < {k}; j++) {{
            if (k_distances[j] > k_distances[max_idx]) {{
                max_idx = j;
            }}
        }}

        // Replace if current is smaller
        if (distances[i] < k_distances[max_idx]) {{
            k_indices[max_idx] = i;
            k_distances[max_idx] = distances[i];
        }}
    }}

    // Vote among k neighbors
    int votes[{n_classes}] = {{0}};
    for (int i = 0; i < {k}; i++) {{
        int label = {safe_name}_training_labels[k_indices[i]];
        for (int c = 0; c < {n_classes}; c++) {{
            if (label == c) {{
                votes[c]++;
                break;
            }}
        }}
    }}

    // Find class with most votes
    int predicted = 0;
    int max_votes = votes[0];
    for (int c = 1; c < {n_classes}; c++) {{
        if (votes[c] > max_votes) {{
            max_votes = votes[c];
            predicted = c;
        }}
    }}

    static const int class_labels[{n_classes}] = {{{', '.join(map(str, model.classes_))}}};
    return class_labels[predicted];
}}
""")

    return ''.join(code_parts)

def generate_discriminant_c_code(model, model_name, scaler, n_features, n_classes):
    """Tạo code C cho LDA/QDA"""
    safe_name = model_name.replace('-', '_').replace(' ', '_')
    is_lda = 'Linear' in model_name or 'LDA' in model_name

    code_parts = []
    code_parts.append(f"""
// ========================================
// {model_name}
// ========================================

""")

    # Scaler
    code_parts.append(f"static const float {safe_name}_scaler_mean[{n_features}] = {{\n")
    for i, val in enumerate(scaler.mean_):
        code_parts.append(f"    {val:.10f}f{',' if i < len(scaler.mean_)-1 else ''}\n")
    code_parts.append("};\n\n")

    code_parts.append(f"static const float {safe_name}_scaler_scale[{n_features}] = {{\n")
    for i, val in enumerate(scaler.scale_):
        code_parts.append(f"    {val:.10f}f{',' if i < len(scaler.scale_)-1 else ''}\n")
    code_parts.append("};\n\n")

    # Class means
    code_parts.append(f"static const float {safe_name}_means[{n_classes}][{n_features}] = {{\n")
    for i in range(n_classes):
        code_parts.append("    {")
        for j in range(n_features):
            code_parts.append(f"{model.means_[i][j]:.10f}f{',' if j < n_features-1 else ''}")
        code_parts.append(f"}}{',' if i < n_classes-1 else ''}\n")
    code_parts.append("};\n\n")

    # Priors
    code_parts.append(f"static const float {safe_name}_priors[{n_classes}] = {{\n")
    for i, val in enumerate(model.priors_):
        code_parts.append(f"    {val:.10f}f{',' if i < len(model.priors_)-1 else ''}\n")
    code_parts.append("};\n\n")

    # Predict function - simplified using Mahalanobis distance
    code_parts.append(f"""
int predict_{safe_name}(float features[{n_features}]) {{
    // Apply scaling
    float features_scaled[{n_features}];
    for (int i = 0; i < {n_features}; i++) {{
        features_scaled[i] = (features[i] - {safe_name}_scaler_mean[i]) / {safe_name}_scaler_scale[i];
    }}

    // Calculate discriminant scores for each class
    float scores[{n_classes}];

    for (int c = 0; c < {n_classes}; c++) {{
        // Simple Euclidean distance to class mean
        float dist = 0.0f;
        for (int i = 0; i < {n_features}; i++) {{
            float diff = features_scaled[i] - {safe_name}_means[c][i];
            dist += diff * diff;
        }}

        // Discriminant score = -distance + log(prior)
        scores[c] = -dist + logf({safe_name}_priors[c]);
    }}

    // Find class with highest score
    int predicted = 0;
    float max_score = scores[0];
    for (int c = 1; c < {n_classes}; c++) {{
        if (scores[c] > max_score) {{
            max_score = scores[c];
            predicted = c;
        }}
    }}

    static const int class_labels[{n_classes}] = {{{', '.join(map(str, model.classes_))}}};
    return class_labels[predicted];
}}
""")

    return ''.join(code_parts)

def generate_naive_bayes_c_code(model, model_name, scaler, n_features, n_classes, debug_log_path=None):
    """Tạo code C cho Naive Bayes"""
    safe_name = model_name.replace('-', '_').replace(' ', '_')
    is_gaussian = 'Gaussian' in model_name

    def log(msg):
        if debug_log_path:
            with open(debug_log_path, 'a', encoding='utf-8') as f:
                f.write(msg + '\n')

    code_parts = []
    code_parts.append(f"""
// ========================================
// {model_name}
// ========================================

""")

    # Scaler
    code_parts.append(f"static const float {safe_name}_scaler_mean[{n_features}] = {{\n")
    for i, val in enumerate(scaler.mean_):
        code_parts.append(f"    {val:.10f}f{',' if i < len(scaler.mean_)-1 else ''}\n")
    code_parts.append("};\n\n")

    code_parts.append(f"static const float {safe_name}_scaler_scale[{n_features}] = {{\n")
    for i, val in enumerate(scaler.scale_):
        code_parts.append(f"    {val:.10f}f{',' if i < len(scaler.scale_)-1 else ''}\n")
    code_parts.append("};\n\n")

    # Class priors - GaussianNB có class_prior_, BernoulliNB có class_log_prior_
    class_log_prior = getattr(model, 'class_log_prior_', None)
    if class_log_prior is None:
        # GaussianNB: tính log từ class_prior_
        class_prior = model.class_prior_
        class_log_prior = np.log(class_prior)
        log(f"GaussianNB: Computing log prior from class_prior_")

    code_parts.append(f"static const float {safe_name}_class_log_prior[{n_classes}] = {{\n")
    for i, val in enumerate(class_log_prior):
        code_parts.append(f"    {val:.10f}f{',' if i < len(class_log_prior)-1 else ''}\n")
    code_parts.append("};\n\n")

    if is_gaussian:
        # Gaussian: store theta (mean) and var (variance)
        # Note: GaussianNB stores means in theta_ and variances in var_ or sigma_
        log(f"DEBUG GaussianNB - model_name: {model_name}")
        log(f"Has theta_: {hasattr(model, 'theta_')}")
        log(f"Has var_: {hasattr(model, 'var_')}")
        log(f"Has sigma_: {hasattr(model, 'sigma_')}")
        log(f"Has epsilon_: {hasattr(model, 'epsilon_')}")

        # Liệt kê TẤT CẢ attributes có trailing underscore (fitted attributes)
        fitted_attrs = [attr for attr in dir(model) if attr.endswith('_') and not attr.startswith('_')]
        log(f"All fitted attributes: {fitted_attrs}")

        theta = getattr(model, 'theta_', None)
        var = getattr(model, 'var_', getattr(model, 'sigma_', None))

        if theta is None:
            raise AttributeError(f"GaussianNB model '{model_name}' missing theta_ attribute. Available: {fitted_attrs}")
        if var is None:
            # Try epsilon_ as fallback (some sklearn versions)
            var = getattr(model, 'epsilon_', None)
            if var is None:
                raise AttributeError(f"GaussianNB model '{model_name}' missing var_/sigma_/epsilon_ attribute. Available: {fitted_attrs}")

        log(f"theta shape: {theta.shape if theta is not None else 'None'}")
        log(f"var shape: {var.shape if var is not None else 'None'}")

        code_parts.append(f"static const float {safe_name}_theta[{n_classes}][{n_features}] = {{\n")
        for i in range(n_classes):
            code_parts.append("    {")
            for j in range(n_features):
                code_parts.append(f"{theta[i][j]:.10f}f{',' if j < n_features-1 else ''}")
            code_parts.append(f"}}{',' if i < n_classes-1 else ''}\n")
        code_parts.append("};\n\n")

        code_parts.append(f"static const float {safe_name}_var[{n_classes}][{n_features}] = {{\n")
        for i in range(n_classes):
            code_parts.append("    {")
            for j in range(n_features):
                code_parts.append(f"{var[i][j]:.10f}f{',' if j < n_features-1 else ''}")
            code_parts.append(f"}}{',' if i < n_classes-1 else ''}\n")
        code_parts.append("};\n\n")

        code_parts.append(f"""
static inline float {safe_name}_gaussian_log_prob(float x, float mean, float var) {{
    const float PI = 3.14159265359f;
    float diff = x - mean;
    return -0.5f * logf(2.0f * PI * var) - (diff * diff) / (2.0f * var);
}}
""")
    else:
        # Bernoulli: store theta (feature log probabilities)
        code_parts.append(f"static const float {safe_name}_theta[{n_classes}][{n_features}] = {{\n")
        for i in range(n_classes):
            code_parts.append("    {")
            for j in range(n_features):
                # BernoulliNB uses feature_log_prob_
                theta_val = model.feature_log_prob_[i][j] if hasattr(model, 'feature_log_prob_') else model.theta_[i][j]
                code_parts.append(f"{theta_val:.10f}f{',' if j < n_features-1 else ''}")
            code_parts.append(f"}}{',' if i < n_classes-1 else ''}\n")
        code_parts.append("};\n\n")

    # Predict function
    code_parts.append(f"""
int predict_{safe_name}(float features[{n_features}]) {{
    // Apply scaling
    float features_scaled[{n_features}];
    for (int i = 0; i < {n_features}; i++) {{
        features_scaled[i] = (features[i] - {safe_name}_scaler_mean[i]) / {safe_name}_scaler_scale[i];
    }}

    // Calculate log probability for each class
    float log_probs[{n_classes}];

    for (int c = 0; c < {n_classes}; c++) {{
        log_probs[c] = {safe_name}_class_log_prior[c];

        for (int i = 0; i < {n_features}; i++) {{
""")

    if is_gaussian:
        code_parts.append(f"""            log_probs[c] += {safe_name}_gaussian_log_prob(
                features_scaled[i],
                {safe_name}_theta[c][i],
                {safe_name}_var[c][i]
            );
""")
    else:
        # Bernoulli - use log probabilities directly
        code_parts.append(f"""            // Bernoulli: use feature log probabilities
            if (features_scaled[i] > 0.5f) {{
                log_probs[c] += {safe_name}_theta[c][i];  // log P(x_i=1|y=c)
            }} else {{
                log_probs[c] += logf(1.0f - expf({safe_name}_theta[c][i]) + 1e-10f);  // log P(x_i=0|y=c)
            }}
""")

    code_parts.append(f"""        }}
    }}

    // Find class with highest log probability
    int predicted = 0;
    float max_log_prob = log_probs[0];
    for (int c = 1; c < {n_classes}; c++) {{
        if (log_probs[c] > max_log_prob) {{
            max_log_prob = log_probs[c];
            predicted = c;
        }}
    }}

    static const int class_labels[{n_classes}] = {{{', '.join(map(str, model.classes_))}}};
    return class_labels[predicted];
}}
""")

    return ''.join(code_parts)


def print_memory_report(trained_models):
    """⚠️ Memory không thể ước tính chính xác từ Python!
    Compile example trong Arduino IDE để xem Flash/RAM thực tế.
    """
    print("\n" + "=" * 100)
    print("TRAINED MODELS SUMMARY")
    print("=" * 100)
    print(f"Tổng số models: {len(trained_models)}")
    print("\nDanh sách models:")
    for i, model_name in enumerate(trained_models.keys(), 1):
        print(f"  {i}. {model_name}")

    print("\n" + "=" * 100)
    print("⚠️  ĐỂ ĐO KÍCH THƯỚC THỰC TẾ:")
    print("\n1. Mở Arduino IDE")
    print("2. Mở: File → Examples → MLPredictor → SerialPredict (hoặc SensorPredict)")
    print("3. Chọn board ESP32/ESP32-S3/v.v...")
    print("4. Compile (Verify) để xem Flash/RAM usage thực tế trong output")
    print("\n💡 Compiler output sẽ hiển thị:")
    print("   - Sketch uses XXXXX bytes (XX%) of program storage space (Flash)")
    print("   - Global variables use XXXX bytes (XX%) of dynamic memory (RAM)")
    print("   - Thời gian thực thi của từng model")
    print("=" * 100 + "\n")

def create_memory_readme(lib_dir, model_c_codes, n_features, n_classes):
    """Tạo README.md với hướng dẫn đo memory (chỉ dùng models đã generate)"""

    readme_content = f"""# {ARDUINO_LIB_NAME}

Auto-generated Machine Learning library for Arduino/ESP32
Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## 📊 Library Information

- **Number of Models**: {len(model_c_codes)}
- **Number of Features**: {n_features}
- **Number of Classes**: {n_classes}

## ⚠️ Memory Measurement

**Python CANNOT accurately calculate Flash/RAM usage!**

To measure real memory usage:

1. Open Arduino IDE
2. Open: `File → Examples → {ARDUINO_LIB_NAME} → SerialPredict` (or SensorPredict)
3. Select your board (ESP32/ESP32-S3/etc.)
4. Click **Verify** to compile and check output for Flash/RAM:
   ```
   Sketch uses XXXXX bytes (XX%) of program storage space
   Global variables use XXXX bytes (XX%) of dynamic memory
   ```
6. Upload and open Serial Monitor (115200) for detailed report

## 📋 Models Included

"""
    for i, model_name in enumerate(model_c_codes.keys(), 1):
        readme_content += f"{i}. {model_name}\n"

    readme_content += f"""

## 🔌 Target Microcontrollers

| MCU | Flash | RAM | Notes |
|-----|-------|-----|-------|
"""

    # Simplified MCU list
    mcu_list = [
        ("Arduino Uno", "32 KB", "2 KB", "Too small for ML"),
        ("Arduino Mega 2560", "256 KB", "8 KB", "May fit small models"),
        ("ESP8266", "4 MB", "80 KB", "Good for ML"),
        ("ESP32", "4-16 MB", "520 KB", "Excellent for ML"),
        ("ESP32-S2", "4-8 MB", "320 KB", "Good for ML"),
        ("ESP32-S3", "4-16 MB", "512 KB", "Excellent for ML"),
        ("ESP32-C3", "4-8 MB", "400 KB", "Good for ML"),
        ("Raspberry Pi Pico", "2 MB", "264 KB", "Good for ML"),
        ("STM32F4", "512 KB-2 MB", "128-256 KB", "Good for ML"),
        ("STM32H7", "1-2 MB", "512 KB-1 MB", "Excellent for ML"),
    ]

    for mcu_name, flash, ram, notes in mcu_list:
        readme_content += f"| {mcu_name} | {flash} | {ram} | {notes} |\n"

    readme_content += f"""

**Compile your sketch to check if your board has enough space!**

## 📝 Usage Example

```cpp
#include <{ARDUINO_LIB_NAME}.h>

float features[NUM_FEATURES];

void setup() {{
    Serial.begin(115200);

    // Print feature information
    print_feature_info();

    // Set your feature values
    features[0] = value1;
    features[1] = value2;
    // ... set all {n_features} features

    // Make prediction (using first model as example)
"""

    first_model = list(model_c_codes.keys())[0]
    safe_first = first_model.replace('-', '_').replace(' ', '_')

    readme_content += f"""    int prediction = predict_{safe_first}(features);

    Serial.print("Predicted class: ");
    Serial.println(prediction);
}}

void loop() {{
    // Your code here
}}
```

## 🎯 Available Models

"""

    for i, model_name in enumerate(model_c_codes.keys(), 1):
        safe_name = model_name.replace('-', '_').replace(' ', '_')
        readme_content += f"{i}. `predict_{safe_name}()` - {model_name}\n"

    readme_content += f"""

## ⚠️ Important Notes

1. **Feature Order**: Features must be provided in the exact order specified
2. **Data Type**: All features must be `float` (32-bit)
3. **Scaling**: StandardScaler is applied automatically inside the library
4. **Memory**: Compile your sketch to see actual Flash/RAM usage
5. **Concurrency**: Models can be called sequentially or use voting

## � ROM Optimization

**Separate Model Files Mode** {'(ENABLED ✅)' if SEPARATE_MODEL_FILES else '(DISABLED ⚠️)'}

This library generates each model as a **separate .cpp file** in the `src/` folder.

**Benefits:**
- ✅ **Linker only compiles models you actually use**
- ✅ If you call 2 models → only those 2 are compiled into binary
- ✅ Significantly reduces Flash/ROM usage on small MCUs

**Example:**
```cpp
// sketch.ino - Only use 2 models
#include <MLPredictor.h>

void setup() {{
    // Only these 2 functions are called
    int pred1 = predict_SVM_rbf_ovo(features);
    int pred2 = predict_Tree_DecisionTree(features);

    // Other 8 models are NOT compiled → Save ROM!
}}
```

**Result:** Binary size ≈ size of 2 models only (typically 20-40 KB)

**To verify:** After compiling, check Arduino IDE output:
```
Sketch uses XXXXX bytes (XX%) of program storage space
```

## �📚 See Examples

Check the `examples/` folder for:
- `SerialPredict/` - Read data from Serial and predict
- `SensorPredict/` - Read from sensors and predict

---
*Auto-generated by ML Pipeline*
"""

    with open(f"{lib_dir}/README.md", 'w', encoding='utf-8') as f:
        f.write(readme_content)
    print(f"✓ Đã tạo: README.md (với memory information)")

def generate_arduino_library(trained_models, feature_names, class_labels, X_train, X_test):
    """Tạo thư viện Arduino/ESP32"""
    print("\n" + "=" * 80)
    print("TẠO THƯ VIỆN ARDUINO/ESP32")
    print("=" * 80)

    lib_dir = f"{OUTPUT_DIR}/{ARDUINO_LIB_NAME}"
    os.makedirs(lib_dir, exist_ok=True)

    n_features = len(feature_names)
    n_classes = len(class_labels)

    # Dictionary để lưu C code và memory info
    model_c_codes = {}
    memory_info = {}

    # Tính value ranges từ data
    value_ranges = {}
    for i, fname in enumerate(feature_names):
        value_ranges[fname] = {
            'min': float(min(X_train[:, i].min(), X_test[:, i].min())),
            'max': float(max(X_train[:, i].max(), X_test[:, i].max()))
        }

    # ========================================
    # BƯỚC 1: Generate C code cho tất cả models
    # ========================================
    print("\n🔧 Generating C code for models...")

    source_content = f"""/*
 * {ARDUINO_LIB_NAME} Implementation
 * Auto-generated on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
 */

#include "{ARDUINO_LIB_NAME}.h"
#include <math.h>

// ========================================
// HELPER FUNCTIONS
// ========================================

void print_feature_info() {{
    Serial.println("\\n========================================");
    Serial.println("FEATURE INFORMATION");
    Serial.println("========================================");
    Serial.print("Number of features: ");
    Serial.println(NUM_FEATURES);
    Serial.print("Number of classes: ");
    Serial.println(NUM_CLASSES);
    Serial.println("\\nFeature order and expected ranges:");

    for (int i = 0; i < NUM_FEATURES; i++) {{
        Serial.print("  ");
        Serial.print(i + 1);
        Serial.print(". ");
        Serial.print(FEATURE_NAMES[i]);
        Serial.print(": [");
        Serial.print(FEATURE_RANGES[i][0], 4);
        Serial.print(", ");
        Serial.print(FEATURE_RANGES[i][1], 4);
        Serial.println("]");
    }}

    Serial.println("\\nClass labels:");
    for (int i = 0; i < NUM_CLASSES; i++) {{
        Serial.print("  ");
        Serial.println(CLASS_LABELS[i]);
    }}
    Serial.println("========================================\\n");
}}

"""

    # Tạo file log để debug (người dùng có thể xem sau)
    debug_log_path = f"{OUTPUT_DIR}/generation_debug.log"
    with open(debug_log_path, 'w', encoding='utf-8') as debug_log:
        debug_log.write(f"=== GENERATION DEBUG LOG ===\n")
        debug_log.write(f"Time: {datetime.now()}\n")
        debug_log.write(f"Models to generate: {list(trained_models.keys())}\n\n")

    for model_name, model_data in trained_models.items():
        try:
            model = model_data['model']
            scaler = model_data['scaler']
            family = model_data.get('family', 'SVM')

            print(f"  → Generating code for {model_name} ({family})...")

            # Log vào file
            with open(debug_log_path, 'a', encoding='utf-8') as debug_log:
                debug_log.write(f"\n{'='*60}\n")
                debug_log.write(f"Processing: {model_name}\n")
                debug_log.write(f"Family: {family}\n")
                debug_log.write(f"Type: {type(model).__name__}\n")

            # Chọn generator phù hợp với loại model
            if family == 'SVM':
                c_code = generate_svm_c_code(model, model_name, scaler, n_features, n_classes)
            elif family == 'Tree':
                c_code = generate_tree_c_code(model, model_name, scaler, n_features, n_classes)
            elif family == 'Neural Network':
                c_code = generate_mlp_c_code(model, model_name, scaler, n_features, n_classes)
            elif family == 'KNN':
                c_code = generate_knn_c_code(model, model_name, scaler, n_features, n_classes)
            elif family == 'Discriminant':
                c_code = generate_discriminant_c_code(model, model_name, scaler, n_features, n_classes)
            elif family == 'Naive Bayes':
                with open(debug_log_path, 'a', encoding='utf-8') as debug_log:
                    debug_log.write(f"Calling generate_naive_bayes_c_code...\n")
                c_code = generate_naive_bayes_c_code(model, model_name, scaler, n_features, n_classes, debug_log_path)
                with open(debug_log_path, 'a', encoding='utf-8') as debug_log:
                    debug_log.write(f"✓ Code generated successfully, length: {len(c_code)}\n")
            else:
                print(f"⚠️  Unknown family '{family}' for {model_name}, SKIPPING (không tạo code)...")
                continue  # Bỏ qua hoàn toàn - không thêm vào model_c_codes và memory_info

            # Lưu C code
            model_c_codes[model_name] = c_code
            source_content += c_code + "\n"

            # ⚠️ Không tính memory từ Python - không chính xác!
            # Compile sketch để đo thực tế
            memory_info[model_name] = {
                'rom_bytes': 0,  # Xem trong compile output
                'ram_bytes': 0,  # Xem trong compile output
                'rom_kb': 0,
                'ram_kb': 0,
                'details': 'Compile sketch to measure'
            }

        except Exception as e:
            error_msg = f"⚠️  Lỗi khi tạo code cho {model_name}: {e}\n"
            error_msg += f"     → Model family: {family}\n"
            error_msg += f"     → Model type: {type(model).__name__}\n"
            print(error_msg)

            # Log chi tiết vào file
            import traceback
            with open(debug_log_path, 'a', encoding='utf-8') as debug_log:
                debug_log.write(f"\n❌ ERROR:\n")
                debug_log.write(error_msg)
                debug_log.write(f"\nFull traceback:\n")
                debug_log.write(traceback.format_exc())
                debug_log.write(f"\n{'='*60}\n")

            traceback.print_exc()
            # SKIP hoàn toàn khi lỗi - không thêm vào model_c_codes hay memory_info
            continue

    # Kiểm tra số lượng models thực tế vs đã chọn
    num_selected = len(trained_models)
    num_generated = len(model_c_codes)

    if num_generated < num_selected:
        print(f"\n⚠️  CẢNH BÁO: Chọn {num_selected} models nhưng chỉ tạo được {num_generated} models!")
        print(f"   → {num_selected - num_generated} model(s) bị skip do lỗi generate code")
        skipped_models = set(trained_models.keys()) - set(model_c_codes.keys())
        for m in skipped_models:
            print(f"      • {m}")

    print(f"\n✅ Tổng cộng: {num_generated}/{num_selected} models được tạo thành công")

    # ========================================
    # BƯỚC 2: Tạo header file (SAU KHI đã có model_c_codes)
    # ========================================
    print("\n📝 Creating header file...")

    feature_ranges = '\n'.join([f"//   {i+1}. {fname}: [{value_ranges[fname]['min']:.4f}, {value_ranges[fname]['max']:.4f}]"
                                for i, fname in enumerate(feature_names)])

    header_content = f"""/*
 * {ARDUINO_LIB_NAME} - Machine Learning Predictor Library
 * Auto-generated on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
 *
 * This library contains ML models for prediction on Arduino/ESP32
 *
 * IMPORTANT: DATA FORMAT REQUIREMENTS
 * ===================================
 * Number of features: {n_features}
 * Number of classes: {n_classes}
 * Classes: {', '.join(map(str, class_labels))}
 *
 * Feature names (IN THIS EXACT ORDER):
{chr(10).join([f' *   {i+1}. {fname}' for i, fname in enumerate(feature_names)])}
 *
 * Expected value ranges (from training data):
{feature_ranges}
 *
 * Data type: float (32-bit floating point)
 * Scaling: Automatic (StandardScaler applied internally)
 *
 * INPUT FORMAT:
 * - Array of {n_features} float values
 * - Must be in the EXACT order listed above
 * - Supports decimal point (.) and negative numbers
 * - Example: float features[{n_features}] = {{value1, value2, ..., value{n_features}}};
 *
 * WARNING: Incorrect order or data type will produce incorrect predictions!
 */

#ifndef {ARDUINO_LIB_NAME.upper()}_H
#define {ARDUINO_LIB_NAME.upper()}_H

#include <Arduino.h>
#include <math.h>

// Model count
#define NUM_MODELS {len(model_c_codes)}
#define NUM_FEATURES {n_features}
#define NUM_CLASSES {n_classes}

// Class labels
static const int CLASS_LABELS[NUM_CLASSES] = {{{', '.join(map(str, class_labels))}}};

// Feature names for reference
static const char* FEATURE_NAMES[NUM_FEATURES] = {{{', '.join([f'"{fname}"' for fname in feature_names])}}};

// Expected value ranges (min, max) for each feature
static const float FEATURE_RANGES[NUM_FEATURES][2] = {{
{chr(10).join([f'  {{{value_ranges[fname]["min"]:.6f}f, {value_ranges[fname]["max"]:.6f}f}}, // {fname}' for fname in feature_names])}
}};

// Helper function to print feature info
void print_feature_info();

"""

    # Tạo function declarations với memory info
    header_content += "\n// ========================================\n"
    header_content += "// MODEL FUNCTIONS\n"
    header_content += "// ========================================\n"

    for model_name in model_c_codes.keys():
        # Kiểm tra xem model có trong memory_info không (defensive programming)
        if model_name not in memory_info:
            print(f"⚠️  Warning: {model_name} không có memory info, bỏ qua")
            continue

        safe_name = model_name.replace('-', '_').replace(' ', '_')
        mem = memory_info[model_name]
        header_content += f"\n// {model_name}\n"
        header_content += f"// {mem['details']}\n"
        header_content += f"int predict_{safe_name}(float features[NUM_FEATURES]);\n"

    header_content += "\n#endif\n"

    # Lưu header
    with open(f"{lib_dir}/{ARDUINO_LIB_NAME}.h", 'w') as f:
        f.write(header_content)
    print(f"✓ Đã tạo: {ARDUINO_LIB_NAME}.h")

    # ========================================
    # BƯỚC 3: Lưu source files
    # ========================================
    if SEPARATE_MODEL_FILES:
        # MODE 1: Mỗi model 1 file riêng (KHUYẾN KHÍCH - tối ưu ROM)
        print(f"\n📁 Separate files mode: Tạo {len(model_c_codes)} file .cpp riêng biệt")
        src_dir = f"{lib_dir}/src"
        os.makedirs(src_dir, exist_ok=True)

        # Tạo main .cpp chỉ chứa helper functions
        main_content = f"""/*
 * {ARDUINO_LIB_NAME} Implementation
 * Auto-generated on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
 */

#include "{ARDUINO_LIB_NAME}.h"
#include <math.h>

// ========================================
// HELPER FUNCTIONS
// ========================================

void print_feature_info() {{
    Serial.println("\\n========================================");
    Serial.println("FEATURE INFORMATION");
    Serial.println("========================================");
    Serial.print("Number of features: ");
    Serial.println(NUM_FEATURES);
    Serial.print("Number of classes: ");
    Serial.println(NUM_CLASSES);
    Serial.println("\\nFeature order and expected ranges:");

    for (int i = 0; i < NUM_FEATURES; i++) {{
        Serial.print("  ");
        Serial.print(i + 1);
        Serial.print(". ");
        Serial.print(FEATURE_NAMES[i]);
        Serial.print(": [");
        Serial.print(FEATURE_RANGES[i][0], 4);
        Serial.print(", ");
        Serial.print(FEATURE_RANGES[i][1], 4);
        Serial.println("]");
    }}

    Serial.println("\\nClass labels:");
    for (int i = 0; i < NUM_CLASSES; i++) {{
        Serial.print("  ");
        Serial.println(CLASS_LABELS[i]);
    }}
    Serial.println("========================================\\n");
}}
"""
        with open(f"{lib_dir}/{ARDUINO_LIB_NAME}.cpp", 'w') as f:
            f.write(main_content)
        print(f"✓ Đã tạo: {ARDUINO_LIB_NAME}.cpp (helper functions)")

        # Tạo file riêng cho từng model
        for model_name, c_code in model_c_codes.items():
            # Kiểm tra xem model có trong memory_info không
            if model_name not in memory_info:
                print(f"⚠️  Warning: {model_name} không có memory info, sử dụng giá trị mặc định")
                memory_info[model_name] = {
                    'rom_bytes': 5000,
                    'ram_bytes': 500,
                    'rom_kb': 5.0,
                    'ram_kb': 0.5,
                    'details': 'Unknown'
                }

            safe_name = model_name.replace('-', '_').replace(' ', '_')
            model_file_content = f"""/*
 * {model_name} Implementation
 * Auto-generated on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
 *
 * ROM Usage: ~{memory_info[model_name]['rom_kb']:.2f} KB
 * RAM Usage: ~{memory_info[model_name]['ram_kb']:.2f} KB
 */

#include "{ARDUINO_LIB_NAME}.h"
#include <math.h>

{c_code}
"""
            with open(f"{src_dir}/{safe_name}.cpp", 'w') as f:
                f.write(model_file_content)
            print(f"  ✓ {safe_name}.cpp ({memory_info[model_name]['rom_kb']:.1f} KB)")

        print(f"\n💡 OPTIMIZATION: Linker sẽ chỉ compile models được gọi trong sketch!")
        print(f"   Ví dụ: Nếu chỉ gọi 2/10 models → Binary size chỉ tăng ~{sum([memory_info[m]['rom_bytes'] for m in list(model_c_codes.keys())[:2]])/1024:.1f} KB")

    else:
        # MODE 2: Tất cả trong 1 file (đơn giản nhưng không tối ưu)
        print(f"\n📄 Single file mode: Tất cả models trong 1 file .cpp")
        with open(f"{lib_dir}/{ARDUINO_LIB_NAME}.cpp", 'w') as f:
            f.write(source_content)
        print(f"✓ Đã tạo: {ARDUINO_LIB_NAME}.cpp")
        print(f"⚠️  WARNING: Compiler có thể compile CẢ {len(model_c_codes)} models ngay cả khi chỉ dùng 1-2 models")
        print(f"   → Đặt SEPARATE_MODEL_FILES = True để tối ưu ROM usage")

    # Tạo README.md với memory information
    create_memory_readme(lib_dir, model_c_codes, n_features, n_classes)

    # In memory report
    print_memory_report(trained_models)

    # Tạo library.properties
    properties_content = f"""name={ARDUINO_LIB_NAME}
version=1.0.0
author=Auto-generated
maintainer=Auto-generated
sentence=Machine Learning prediction library for Arduino/ESP32
paragraph=SVM models for classification
category=Data Processing
url=
architectures=*
includes={ARDUINO_LIB_NAME}.h
"""

    with open(f"{lib_dir}/library.properties", 'w') as f:
        f.write(properties_content)
    print(f"✓ Đã tạo: library.properties")

    # Tạo examples (chỉ dùng models đã generate thành công)
    create_arduino_examples(lib_dir, model_c_codes, feature_names, n_features, n_classes)

    return lib_dir

def create_arduino_examples(lib_dir, model_c_codes, feature_names, n_features, n_classes):
    """Tạo file examples cho Arduino (chỉ dùng models đã generate C code)"""
    examples_dir = f"{lib_dir}/examples"
    os.makedirs(examples_dir, exist_ok=True)

    # Example 1: Serial Input/Output
    example1_dir = f"{examples_dir}/SerialPredict"
    os.makedirs(example1_dir, exist_ok=True)

    model_list = '\n   '.join([f"// {i+1}. {name}" for i, name in enumerate(model_c_codes.keys())])
    first_model = list(model_c_codes.keys())[0]
    safe_first = first_model.replace('-', '_').replace(' ', '_')

    example1_content = f"""/*
 * Serial Prediction Example
 *
 * Nhập dữ liệu qua Serial Monitor và nhận kết quả prediction
 * Format: {', '.join([f'feature{i+1}' for i in range(n_features)])}
 * Ví dụ: 1.5, 2.3, 4.1
 *
 * Available models:
   {model_list}
 */

#include <{ARDUINO_LIB_NAME}.h>

float features[NUM_FEATURES];

void setup() {{
    Serial.begin(115200);
    while (!Serial) delay(10);

    Serial.println("========================================");
    Serial.println("ML Predictor - Serial Mode");
    Serial.println("========================================");

    // Print feature information
    print_feature_info();

    Serial.println("\\nPaste {n_features} values from Excel (TAB/space separated):");
    Serial.println("Example: 242,1875\\t529,291748\\t-92,64015961...");
    Serial.println("(Use comma for decimal, paste directly from Excel)");
    Serial.println("Type 'info' to show feature information again\\n");
}}

void loop() {{
    if (Serial.available() > 0) {{
        // Read input
        String input = Serial.readStringUntil('\\n');
        input.trim();

        // Check for info command
        if (input.equalsIgnoreCase("info")) {{
            print_feature_info();
            return;
        }}

        // Convert European/Vietnamese decimal comma (,) to decimal point (.)
        for (int i = 0; i < input.length(); i++) {{
            if (input.charAt(i) == ',') {{
                input.setCharAt(i, '.');
            }}
        }}

        // Parse values separated by TAB or space
        int featureCount = 0;
        int startIdx = 0;

        for (int i = 0; i <= input.length(); i++) {{
            bool isDelimiter = (i == input.length()) ||
                              (input.charAt(i) == '\\t') ||
                              (input.charAt(i) == ' ');

            if (isDelimiter) {{
                if (i > startIdx) {{
                    if (featureCount < NUM_FEATURES) {{
                        String value = input.substring(startIdx, i);
                        value.trim();
                        if (value.length() > 0) {{
                            features[featureCount] = value.toFloat();
                            featureCount++;
                        }}
                    }}
                }}
                startIdx = i + 1;
            }}
        }}

        if (featureCount == NUM_FEATURES) {{
            // Display input
            Serial.print("\\nInput: [");
            for (int i = 0; i < NUM_FEATURES; i++) {{
                Serial.print(FEATURE_NAMES[i]);
                Serial.print("=");
                Serial.print(features[i], 4);
                if (i < NUM_FEATURES - 1) Serial.print(", ");
            }}
            Serial.println("]");

            // Predict using all models
            Serial.println("\\n--- Predictions from all models ---");
{chr(10).join([f'            Serial.print("{name}: Class ");' + chr(10) + f'            Serial.println(predict_{name.replace("-", "_").replace(" ", "_")}(features));' for name in model_c_codes.keys()])}
            Serial.println("----------------------------------------");
            Serial.println("Paste next values:");
        }} else {{
            Serial.println("Error: Need exactly " + String(NUM_FEATURES) + " values!");
            Serial.print("Got ");
            Serial.print(featureCount);
            Serial.print(" values, expected ");
            Serial.println(NUM_FEATURES);
        }}
    }}
}}
"""

    with open(f"{example1_dir}/SerialPredict.ino", 'w') as f:
        f.write(example1_content)
    print(f"✓ Đã tạo: examples/SerialPredict/SerialPredict.ino")

    # Example 2: Sensor Input
    example2_dir = f"{examples_dir}/SensorPredict"
    os.makedirs(example2_dir, exist_ok=True)

    example2_content = f"""/*
 * Sensor Prediction Example
 *
 * Đọc dữ liệu từ cảm biến và thực hiện prediction
 * Thay đổi hàm readSensors() để đọc từ cảm biến thật
 *
 * Available models:
   {model_list}
 */

#include <{ARDUINO_LIB_NAME}.h>

float features[NUM_FEATURES];
unsigned long lastPrediction = 0;
const unsigned long PREDICTION_INTERVAL = 1000; // 1 giây

void setup() {{
    Serial.begin(115200);
    while (!Serial) delay(10);

    Serial.println("========================================");
    Serial.println("ML Predictor - Sensor Mode");
    Serial.println("========================================");

    // Khởi tạo cảm biến của bạn ở đây
    initSensors();
}}

void loop() {{
    unsigned long currentMillis = millis();

    if (currentMillis - lastPrediction >= PREDICTION_INTERVAL) {{
        lastPrediction = currentMillis;

        // Đọc dữ liệu từ cảm biến
        readSensors();

        // Hiển thị dữ liệu
        Serial.print("Sensor data: [");
        for (int i = 0; i < NUM_FEATURES; i++) {{
            Serial.print(features[i], 4);
            if (i < NUM_FEATURES - 1) Serial.print(", ");
        }}
        Serial.println("]");

        // Predict using all models
        Serial.println("--- Predictions ---");
{chr(10).join([f'        Serial.print("{name}: ");' + chr(10) + f'        Serial.println(predict_{name.replace("-", "_").replace(" ", "_")}(features));' for name in model_c_codes.keys()])}
        Serial.println();
    }}
}}

void initSensors() {{
    // TODO: Khởi tạo cảm biến của bạn
    // Ví dụ:
    // pinMode(SENSOR_PIN, INPUT);
    // sensor.begin();

    Serial.println("✓ Sensors initialized");
}}

void readSensors() {{
    // TODO: Đọc dữ liệu từ cảm biến thật
    // Ví dụ với {n_features} features:

    {"".join([f"features[{i}] = analogRead(A{i}) * (5.0 / 1023.0); // Sensor {i+1}\\n    " for i in range(min(n_features, 6))])}

    // Hoặc từ cảm biến I2C/SPI:
    // features[0] = sensor.readTemperature();
    // features[1] = sensor.readHumidity();
    // ...
}}
"""

    with open(f"{example2_dir}/SensorPredict.ino", 'w') as f:
        f.write(example2_content)
    print(f"✓ Đã tạo: examples/SensorPredict/SensorPredict.ino")

    print(f"\\n✅ Đã tạo {2} example files trong thư viện")


# ============================================================================
# PACKAGE AND DOWNLOAD
# ============================================================================

def package_and_download():
    """Nén tất cả và tải xuống"""
    print("\n" + "=" * 80)
    print("NÉN VÀ TẢI XUỐNG")
    print("=" * 80)

    # Tạo file RAR/ZIP
    archive_name = f"{OUTPUT_DIR}_Results"

    try:
        # Sử dụng shutil để tạo zip (RAR yêu cầu WinRAR)
        shutil.make_archive(archive_name, 'zip', OUTPUT_DIR)
        print(f"✓ Đã nén thành: {archive_name}.zip")

        # Tải xuống nếu đang ở Google Colab
        if IN_COLAB:
            files.download(f"{archive_name}.zip")
            print("✓ Đang tải xuống file...")
        else:
            print(f"✓ File đã sẵn sàng tại: {archive_name}.zip")

    except Exception as e:
        print(f"❌ Lỗi khi nén: {e}")

# ============================================================================
# GRADIO WEB INTERFACE
# ============================================================================

# Global variables cho Gradio
gradio_data = {}
gradio_results = {}

def gradio_process_single_file(file, train_ratio, random_state, progress=gr.Progress()):
    """Xử lý file dữ liệu chưa chia cho Gradio"""
    try:
        progress(0.1, desc="📂 Đang tải file...")
        global SINGLE_FILE_PATH, TRAIN_FILE_PATH, TEST_FILE_PATH, TRAIN_RATIO, RANDOM_STATE, gradio_data

        # Xóa thư mục OUTPUT_DIR cũ nếu tồn tại (để bắt đầu mới)
        if os.path.exists(OUTPUT_DIR):
            import shutil
            try:
                shutil.rmtree(OUTPUT_DIR)
                print(f"✓ Đã xóa thư mục cũ: {OUTPUT_DIR}")
            except Exception as e:
                print(f"⚠️ Không thể xóa thư mục cũ: {e}")

        # Lưu file tạm
        progress(0.2, desc="💾 Lưu file tạm thời...")
        SINGLE_FILE_PATH = file.name
        TRAIN_FILE_PATH = ""
        TEST_FILE_PATH = ""
        TRAIN_RATIO = train_ratio
        RANDOM_STATE = int(random_state)

        # Load dữ liệu
        progress(0.4, desc="📊 Đang load và phân tích dữ liệu...")
        X_train, X_test, y_train, y_test, feature_names = load_data()

        # Lưu vào global
        gradio_data['X_train'] = X_train
        gradio_data['X_test'] = X_test
        gradio_data['y_train'] = y_train
        gradio_data['y_test'] = y_test
        gradio_data['feature_names'] = feature_names

        # Tạo thông báo
        class_counts = pd.Series(y_train).value_counts().sort_index()
        class_names = sorted(np.unique(y_train))

        info = f"""✅ **Tải file thành công!**

📊 **Thông tin dữ liệu:**
- Tổng số mẫu: {len(X_train) + len(X_test)}
- Số features: {len(feature_names)}
- Tên features: {', '.join(feature_names)}
- Số classes: {len(class_names)}
- Tên classes: {', '.join(map(str, class_names))}

📈 **Phân bố classes:**
{chr(10).join([f"- Class {cls}: {count} mẫu (train)" for cls, count in class_counts.items()])}

✂️ **Đã chia dữ liệu bằng thuật toán SPXY:**
- Train: {len(X_train)} mẫu ({train_ratio*100:.0f}%)
- Test: {len(X_test)} mẫu ({(1-train_ratio)*100:.0f}%)

---

### 🎉 **XONG BƯỚC 1! HÃY CHUYỂN SANG BƯỚC 2 ĐỂ TRAINING**
"""

        # Vẽ biểu đồ vào file
        os.makedirs(OUTPUT_DIR, exist_ok=True)
        plot_data_distribution(X_train, X_test, y_train, y_test, feature_names)

        plot_path = f"{OUTPUT_DIR}/data_distribution.png"
        return info, plot_path

    except Exception as e:
        return f"❌ Lỗi: {str(e)}", None

def gradio_process_split_files(train_file, test_file, progress=gr.Progress()):
    """Xử lý files đã chia sẵn cho Gradio"""
    try:
        progress(0.1, desc="📂 Đang tải files...")
        global SINGLE_FILE_PATH, TRAIN_FILE_PATH, TEST_FILE_PATH, gradio_data

        # Xóa thư mục OUTPUT_DIR cũ nếu tồn tại (để bắt đầu mới)
        if os.path.exists(OUTPUT_DIR):
            import shutil
            try:
                shutil.rmtree(OUTPUT_DIR)
                print(f"✓ Đã xóa thư mục cũ: {OUTPUT_DIR}")
            except Exception as e:
                print(f"⚠️ Không thể xóa thư mục cũ: {e}")

        progress(0.2, desc="💾 Lưu files tạm thời...")
        SINGLE_FILE_PATH = ""
        TRAIN_FILE_PATH = train_file.name
        TEST_FILE_PATH = test_file.name

        # Load dữ liệu
        progress(0.4, desc="📊 Đang load và kiểm tra dữ liệu...")
        X_train, X_test, y_train, y_test, feature_names = load_data()

        # Lưu vào global
        gradio_data['X_train'] = X_train
        gradio_data['X_test'] = X_test
        gradio_data['y_train'] = y_train
        gradio_data['y_test'] = y_test
        gradio_data['feature_names'] = feature_names

        # Tạo thông báo
        train_counts = pd.Series(y_train).value_counts().sort_index()
        test_counts = pd.Series(y_test).value_counts().sort_index()
        class_names = sorted(np.unique(np.concatenate([y_train, y_test])))

        info = f"""✅ **Tải files thành công!**

📊 **Thông tin dữ liệu:**
- Train: {len(X_train)} mẫu
- Test: {len(X_test)} mẫu
- Số features: {len(feature_names)}
- Tên features: {', '.join(feature_names)}
- Số classes: {len(class_names)}
- Tên classes: {', '.join(map(str, class_names))}

📈 **Phân bố classes trong Train:**
{chr(10).join([f"- Class {cls}: {count} mẫu" for cls, count in train_counts.items()])}

📈 **Phân bố classes trong Test:**
{chr(10).join([f"- Class {cls}: {count} mẫu" for cls, count in test_counts.items()])}

---

### 🎉 **XONG BƯỚC 1! HÃY CHUYỂN SANG BƯỚC 2 ĐỂ TRAINING**
"""

        # Vẽ biểu đồ vào file
        os.makedirs(OUTPUT_DIR, exist_ok=True)
        plot_data_distribution(X_train, X_test, y_train, y_test, feature_names)

        # Trả về cả info và path ảnh
        plot_path = f"{OUTPUT_DIR}/data_distribution.png"
        return info, plot_path

    except Exception as e:
        return f"❌ Lỗi: {str(e)}", None

def gradio_train_models(n_trials, cv_folds, optuna_timeout, output_dir, selected_groups, progress=gr.Progress()):
    """Train models cho Gradio với progress bar"""
    try:
        global N_TRIALS, CV_FOLDS, OPTUNA_TIMEOUT, OUTPUT_DIR, gradio_results

        if not gradio_data:
            return "❌ Chưa có dữ liệu. Vui lòng tải file trước!", None, ""

        # Cập nhật config
        N_TRIALS = int(n_trials)
        CV_FOLDS = int(cv_folds)
        OPTUNA_TIMEOUT = int(optuna_timeout)
        OUTPUT_DIR = output_dir

        # Tạo thư mục nếu chưa tồn tại
        os.makedirs(OUTPUT_DIR, exist_ok=True)

        # Ánh xạ nhóm sang family
        group_to_family = {
            "SVM (9 models)": "SVM",
            "Tree (5 models)": "Tree",
            "Neural Network (3 models)": "Neural Network",
            "KNN (2 models)": "KNN",
            "Discriminant (2 models)": "Discriminant",
            "Naive Bayes (2 models)": "Naive Bayes"
        }

        # Lọc family được chọn
        selected_families = [group_to_family[g] for g in selected_groups if g in group_to_family]

        if not selected_families:
            return "❌ Vui lòng chọn ít nhất 1 nhóm mô hình!", None, ""

        X_train = gradio_data['X_train']
        X_test = gradio_data['X_test']
        y_train = gradio_data['y_train']
        y_test = gradio_data['y_test']

        import time
        start_time = time.time()

        # Tắt optuna logging
        optuna.logging.set_verbosity(optuna.logging.WARNING)

        # Chuẩn hóa dữ liệu
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Lấy tất cả model configs và lọc theo nhóm đã chọn
        all_model_configs = get_model_configs()
        model_configs = {name: config for name, config in all_model_configs.items()
                        if config.get('family') in selected_families}
        total_models = len(model_configs)

        all_results = []
        trained_models = {}
        confusion_matrices = {}

        # Train từng model với progress
        for idx, (model_name, config) in enumerate(model_configs.items()):
            progress_pct = idx / total_models
            progress(progress_pct, desc=f"🤖 Training {model_name} ({idx+1}/{total_models})...")

            try:
                model, results, cm = train_single_model(
                    model_name, config, X_train_scaled, y_train, X_test_scaled, y_test, scaler
                )

                if model is not None and results is not None:
                    all_results.append(results)
                    trained_models[model_name] = {'model': model, 'scaler': scaler, 'family': config.get('family', 'SVM')}
                    confusion_matrices[model_name] = cm
            except Exception as e:
                print(f"❌ Lỗi khi train {model_name}: {e}")
                continue

        progress(0.9, desc="💾 Lưu kết quả...")

        # Lưu kết quả
        save_results(all_results, confusion_matrices)

        # Lưu vào global
        gradio_results['all_results'] = all_results
        gradio_results['trained_models'] = trained_models
        gradio_results['confusion_matrices'] = confusion_matrices

        # Tính thời gian
        elapsed_time = time.time() - start_time
        hours = int(elapsed_time // 3600)
        minutes = int((elapsed_time % 3600) // 60)
        seconds = int(elapsed_time % 60)
        time_str = f"{hours}h {minutes}m {seconds}s" if hours > 0 else f"{minutes}m {seconds}s" if minutes > 0 else f"{seconds}s"

        progress(1.0, desc=f"✅ Hoàn thành! ({time_str})")

        # Tạo DataFrame kết quả - Sửa tên cột đúng
        results_df = pd.DataFrame(all_results)
        results_df = results_df.sort_values('Test Accuracy', ascending=False)

        # Ẩn các cột không cần thiết cho Gradio UI
        columns_to_hide = ['Algorithm Family', 'Kernel/Type', 'Best params (JSON)',
                          'CV Train Recall', 'CV Train F1', 'Test Recall', 'Test F1',
                          'Prediction speed (obs/sec)', 'Model size (bytes)']
        display_df = results_df.drop(columns=[col for col in columns_to_hide if col in results_df.columns], errors='ignore')

        # Copy để format, giữ nguyên bản gốc
        display_df = display_df.copy()

        # Format các cột % cho dễ đọc (nhân 100 vì data ở dạng 0-1)
        for col in display_df.columns:
            if 'Accuracy' in col or 'Precision' in col:
                if col in display_df.columns:
                    display_df[col] = display_df[col].apply(lambda x: f"{x*100:.2f}%" if pd.notna(x) else "")

        # Lưu kết quả training vào 1 file Excel duy nhất
        excel_path = f"{OUTPUT_DIR}/results.xlsx"
        with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
            # Sheet 1: Training results (dạng numeric để sort được)
            results_df.to_excel(writer, sheet_name='Training Results', index=False)
            # Sheet 2: Summary (format % cho dễ đọc)
            display_df.to_excel(writer, sheet_name='Summary', index=False)
        print(f"✓ Đã lưu kết quả: {excel_path}")

        # Tạo summary
        summary = f"""✅ **Training hoàn thành!**

⏱️ **Thời gian:** {time_str}

📊 **Tổng quan:**
- Đã train: {len(all_results)} models
- Đã lưu kết quả vào: {OUTPUT_DIR}/

🏆 **Top 10 Models (theo Test Accuracy):**
"""

        top_10 = results_df.head(10)
        for idx, (i, row) in enumerate(top_10.iterrows(), 1):
            summary += f"\n{idx}. **{row['Model']}**: Acc={row['Test Accuracy']:.2f}%, Prec={row['Test Precision']:.2f}%"

        summary += "\n\n---\n\n### 🎉 **XONG BƯỚC 2! HÃY CHUYỂN SANG BƯỚC 3 ĐỂ CHỌẨN MODELS**\n"

        # Tạo danh sách đầy đủ cho manual selection
        models_list_md = "\n📋 **Danh sách tất cả models (sắp xếp theo Test Accuracy):**\n\n"
        for idx, (i, row) in enumerate(results_df.iterrows(), 1):
            models_list_md += f"{idx}. {row['Model']} - Acc: {row['Test Accuracy']:.2f}%\n"

        return summary, display_df, models_list_md

    except Exception as e:
        import traceback
        return f"❌ Lỗi: {str(e)}\n\n```\n{traceback.format_exc()}\n```", None, ""

def gradio_select_models(selection_mode, model_indices, auto_count):
    """Chọn models cho Gradio"""
    try:
        global gradio_results

        if 'all_results' not in gradio_results:
            return "❌ Chưa có kết quả training!", None

        all_results = gradio_results['all_results']
        trained_models = gradio_results['trained_models']

        selected_models = {}
        num_models = int(auto_count) if auto_count else 10

        if selection_mode == "Top N (Test Accuracy)":
            # Sắp xếp theo Test Accuracy
            sorted_results = sorted(all_results, key=lambda x: x['Test Accuracy'], reverse=True)
            for i in range(min(num_models, len(sorted_results))):
                model_name = sorted_results[i]['Model']
                selected_models[model_name] = trained_models[model_name]

        elif selection_mode == "Top N (Train Accuracy)":
            # Sắp xếp theo CV Train Accuracy
            sorted_results = sorted(all_results, key=lambda x: x['CV Train Accuracy'], reverse=True)
            for i in range(min(num_models, len(sorted_results))):
                model_name = sorted_results[i]['Model']
                selected_models[model_name] = trained_models[model_name]

        elif selection_mode == "Top N (Train Precision)":
            # Sắp xếp theo CV Train Precision
            sorted_results = sorted(all_results, key=lambda x: x['CV Train Precision'], reverse=True)
            for i in range(min(num_models, len(sorted_results))):
                model_name = sorted_results[i]['Model']
                selected_models[model_name] = trained_models[model_name]

        elif selection_mode == "Top N (Test Precision)":
            # Sắp xếp theo Test Precision
            sorted_results = sorted(all_results, key=lambda x: x['Test Precision'], reverse=True)
            for i in range(min(num_models, len(sorted_results))):
                model_name = sorted_results[i]['Model']
                selected_models[model_name] = trained_models[model_name]

        else:  # Manual
            try:
                indices = [int(x.strip()) for x in model_indices.split(',')]
                sorted_results = sorted(all_results, key=lambda x: x['Test Accuracy'], reverse=True)
                for idx in indices:
                    if 1 <= idx <= len(sorted_results):
                        model_name = sorted_results[idx-1]['Model']
                        selected_models[model_name] = trained_models[model_name]
            except:
                return "❌ Lỗi: Nhập số theo format: 1,2,3", None

        if not selected_models:
            return "❌ Không có model nào được chọn!", None

        gradio_results['selected_models'] = selected_models

        info = f"✅ **Đã chọn {len(selected_models)} models:**\n"
        for name in selected_models.keys():
            info += f"- {name}\n"

        info += "\n---\n\n### 🎉 **XONG BƯỚC 3! HÃY CHUYỂN SANG BƯỚC 4 ĐỂ TẠO THƯ VIỆN ARDUINO**\n"

        return info, None

    except Exception as e:
        return f"❌ Lỗi: {str(e)}", None

def gradio_generate_arduino(lib_name, separate_files):
    """Tạo thư viện Arduino cho Gradio"""
    try:
        global ARDUINO_LIB_NAME, SEPARATE_MODEL_FILES, gradio_results

        if 'selected_models' not in gradio_results:
            return "❌ Chưa chọn models!", None

        ARDUINO_LIB_NAME = lib_name
        SEPARATE_MODEL_FILES = separate_files

        selected_models = gradio_results['selected_models']
        X_train = gradio_data['X_train']
        X_test = gradio_data['X_test']
        y_train = gradio_data['y_train']
        feature_names = gradio_data['feature_names']
        class_labels = sorted(np.unique(y_train))

        # Generate Arduino library
        lib_dir = generate_arduino_library(selected_models, feature_names, class_labels, X_train, X_test)

        # ⚠️ Không tính memory từ Python - không chính xác!
        memory_info = {}
        for model_name in selected_models.keys():
            memory_info[model_name] = {
                'rom_bytes': 0,
                'ram_bytes': 0,
                'rom_kb': 0,
                'ram_kb': 0,
                'details': 'Compile to measure'
            }

        # Tạo bảng MCU compatibility
        mcu_table = "\n🔌 **Tương thích vi điều khiển:**\n\n"
        mcu_table += "| Vi điều khiển | Flash (KB) | SRAM (KB) |\n"
        mcu_table += "|--------------|-------|------|\n"

        mcu_specs = [
            ("Arduino Uno", 32, 2),
            ("Arduino Nano", 32, 2),
            ("Arduino Pro Mini", 32, 2),
            ("Arduino Mega 2560", 256, 8),
            ("ESP8266", (1024, 16384), 80),
            ("ESP32", (4096, 16384), 520),
            ("ESP32-S2", (4096, 16384), 320),
            ("ESP32-S3", (4096, 16384), 512),
            ("ESP32-C3", (4096, 16384), 400),
            ("Raspberry Pi Pico", (2048, 16384), 264),
            ("STM32F103", (64, 512), (20, 64)),
            ("STM32F4", (256, 2048), (64, 256)),
            ("STM32F7", (512, 2048), (256, 512)),
            ("STM32H7", (1024, 2048), (512, 1024)),
            ("STM32L", (64, 1024), (16, 320)),
        ]

        for mcu_name, flash, sram in mcu_specs:
            # Xử lý Flash range hoặc giá trị đơn
            if isinstance(flash, tuple):
                flash_min = flash[0]
                flash_str = f"{flash[0]}–{flash[1]} KB"
            else:
                flash_min = flash
                flash_str = f"{flash} KB"

            # Xử lý SRAM range hoặc giá trị đơn
            if isinstance(sram, tuple):
                sram_min = sram[0]
                sram_str = f"{sram[0]}–{sram[1]} KB"
            else:
                sram_min = sram
                sram_str = f"{sram} KB"

            # Chỉ hiển thị kích thước, không đánh giá phù hợp
            mcu_table += f"| {mcu_name} | {flash_str} | {sram_str} |\n"

        # Append MCU compatibility và memory info vào results.xlsx
        excel_path = f"{OUTPUT_DIR}/results.xlsx"
        with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a') as writer:
            # Sheet 5: Library Info (tổng quan)
            lib_info_df = pd.DataFrame([{
                'Library Name': lib_name,
                'Number of Models': len(memory_info),
                'Note': '⚠️ Compile example in Arduino IDE to measure actual Flash/RAM usage'
            }])
            lib_info_df.to_excel(writer, sheet_name='Library Info', index=False)

            # Sheet 6: Model list
            mem_df = pd.DataFrame([
                {'Model': name, 'Note': 'Compile to see size'}
                for name in memory_info.keys()
            ])
            mem_df.to_excel(writer, sheet_name='Model List', index=False)

            # Sheet 7: Hướng dẫn đo memory
            instructions = pd.DataFrame([
                {'Step': 1, 'Action': 'Open Arduino IDE'},
                {'Step': 2, 'Action': 'Open your sketch using the library'},
                {'Step': 3, 'Action': 'Select your board (ESP32/ESP32-S3/...)'},
                {'Step': 4, 'Action': 'Click Verify to see Flash/RAM in compile output'},
                {'Step': 5, 'Action': 'Upload and open Serial Monitor (115200)'},
                {'Step': 6, 'Action': 'View detailed memory report and benchmarks'},
            ])
            instructions.to_excel(writer, sheet_name='How to Measure', index=False)

        print(f"✓ Đã lưu thông tin Arduino vào: {excel_path}")

        # Kiểm tra file debug log
        debug_log_exists = os.path.exists(f"{OUTPUT_DIR}/generation_debug.log")
        debug_note = ""
        if debug_log_exists:
            debug_note = f"""
⚠️ **LƯU Ý**: Có file debug log chi tiết tại:
📄 `{OUTPUT_DIR}/generation_debug.log`
→ Xem file này nếu có model bị skip (không tạo được code)
"""

        info = f"""✅ **Đã tạo thư viện Arduino: {lib_name}**

📦 **Thư viện đã được tạo tại:** `{OUTPUT_DIR}/{lib_name}/`
{debug_note}
⚠️ **ĐỂ ĐO KÍCH THƯỚC THỰC TẾ:**

🔍 **Bước 1: Compile để xem Flash/RAM**
1. Mở Arduino IDE
2. Mở sketch sử dụng thư viện
3. Chọn board (ESP32/ESP32-S3/...)
4. Click **Verify** (Compile)
5. Xem output cuối cùng:
   ```
   Sketch uses XXXXX bytes (XX%) of program storage space
   Global variables use XXXX bytes (XX%) of dynamic memory
   ```

📊 **Bước 2: Upload để xem chi tiết**
6. Upload sketch lên board
7. Mở Serial Monitor (115200 baud)
8. Xem báo cáo đầy đủ:
   - Flash Size thực tế
   - Heap/RAM usage
   - Thời gian thực thi từng model

{mcu_table}

💡 **Các sketch có sẵn:**
1. **SerialPredict**: Đọc dữ liệu từ Serial Monitor
2. **SerialPredict**: Nhập data qua Serial để predict
3. **SensorPredict**: Template cho đọc cảm biến

🔧 **Các models đã chọn ({len(selected_models)}):**
"""
        for name, mem in memory_info.items():
            info += f"- {name}\n"

        info += f"\n📂 **Chế độ:** {'Tách files riêng (tối ưu Flash)' if separate_files else 'File đơn'}\n"
        info += f"📊 **Đã lưu Excel:** {excel_path}\n"
        info += "\n---\n\n### 🎉 **XONG BƯỚC 4! HÃY CHUYỂN SANG BƯỚC 5 ĐỂ TẢI XUỐNG**\n"

        return info, None

    except Exception as e:
        import traceback
        return f"❌ Lỗi: {str(e)}\n\n```\n{traceback.format_exc()}\n```", None

def gradio_create_download():
    """Tự động nén và tải file ZIP"""
    try:
        # Tự động nén tất cả kết quả
        package_and_download()

        zip_file = f"{OUTPUT_DIR}_Results.zip"
        if os.path.exists(zip_file):
            file_size = os.path.getsize(zip_file) / (1024 * 1024)  # MB
            return zip_file, f"✅ Đã nén xong! File: {zip_file} ({file_size:.1f} MB)"
        else:
            return None, "❌ Không tìm thấy file ZIP. Vui lòng chạy Training trước!"

    except Exception as e:
        return None, f"❌ Lỗi khi nén: {str(e)}"

def create_gradio_interface():
    """Tạo giao diện Gradio"""

    # CSS để ẩn footer Gradio và thêm animation cho buttons
    custom_css = """
    footer {display: none !important;}

    /* Card-like sections */
    .gradio-container {
        max-width: 1400px !important;
        margin: auto !important;
    }

    /* Tab styling */
    .tabs {
        border-radius: 8px;
        overflow: hidden;
    }

    /* Better spacing */
    .gradio-row {
        gap: 16px !important;
    }

    .gradio-column {
        padding: 12px !important;
        background: #1e293b !important;
        border-radius: 8px;
    }

    /* Button hover effect - phóng to khi hover */
    .pulse-btn button {
        transition: all 0.3s ease !important;
        font-weight: 600 !important;
        border-radius: 8px !important;
    }
    .pulse-btn button:hover {
        transform: scale(1.05) !important;
        box-shadow: 0 0 20px rgba(99, 102, 241, 0.5) !important;
    }

    /* Loading state - mờ đi khi click */
    .pulse-btn button:active {
        opacity: 0.4 !important;
        transform: scale(0.98) !important;
    }

    /* Link-like help buttons (tiêu đề xanh có thể bấm) */
    .link-btn button {
        background: linear-gradient(135deg, #1e3a8a 0%, #3b82f6 100%) !important;
        color: white !important;
        border: none !important;
        padding: 8px 16px !important;
        border-radius: 6px !important;
        box-shadow: 0 2px 8px rgba(30, 58, 138, 0.3) !important;
        font-weight: 600 !important;
        font-size: 13px !important;
        transition: all 0.3s ease !important;
    }
    .link-btn button:hover {
        background: linear-gradient(135deg, #1e40af 0%, #60a5fa 100%) !important;
        transform: translateY(-2px) !important;
        box-shadow: 0 4px 12px rgba(30, 58, 138, 0.4) !important;
    }

    /* Training config inputs - nhỏ gọn hơn */
    .training-input input,
    .training-input textarea {
        font-size: 14px !important;
        padding: 8px 12px !important;
        background: #1e293b !important;
        border: 1px solid #475569 !important;
        color: #e2e8f0 !important;
        border-radius: 6px !important;
    }

    .training-input input:focus,
    .training-input textarea:focus {
        border-color: #3b82f6 !important;
        box-shadow: 0 0 0 2px rgba(59, 130, 246, 0.2) !important;
    }

    /* Progress status */
    .status-box {
        padding: 16px;
        border-radius: 8px;
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        font-weight: 600;
    }

    /* Markdown headings */
    .markdown-text h3 {
        color: #1f2937;
        border-bottom: 2px solid #e5e7eb;
        padding-bottom: 8px;
        margin-top: 24px;
    }

    /* File upload areas */
    .file-upload {
        border: 2px dashed #475569 !important;
        border-radius: 8px !important;
        padding: 20px !important;
        transition: all 0.3s ease !important;
        background: #0f172a !important;
        color: #94a3b8 !important;
    }

    .file-upload:hover {
        border-color: #64748b !important;
        background: #1e293b !important;
        box-shadow: 0 4px 12px rgba(71, 85, 105, 0.3) !important;
    }

    /* Progress bar - make it LARGE and prominent */
    .progress-container {
        position: fixed !important;
        top: 50% !important;
        left: 50% !important;
        transform: translate(-50%, -50%) !important;
        z-index: 9999 !important;
        background: rgba(0, 0, 0, 0.85) !important;
        padding: 40px 60px !important;
        border-radius: 16px !important;
        box-shadow: 0 20px 60px rgba(0, 0, 0, 0.5) !important;
    }

    .progress-text {
        color: white !important;
        font-size: 24px !important;
        font-weight: 700 !important;
        text-align: center !important;
        margin-bottom: 20px !important;
    }

    .progress-bar {
        height: 12px !important;
        border-radius: 6px !important;
        background: linear-gradient(90deg, #3b82f6, #8b5cf6) !important;
    }
    """

    with gr.Blocks(title="ML Pipeline - Arduino Library Generator", theme=gr.themes.Soft(), css=custom_css) as app:
        gr.Markdown("""
        # 🤖 Machine Learning Pipeline - Arduino Library Generator
        ### Train ML models và tạo thư viện cho Arduino/ESP32
        ---
        """)

        with gr.Tabs():
            # ===== TAB 1: UPLOAD DATA =====
            with gr.Tab("📁 1. Tải Dữ Liệu"):
                gr.Markdown("""
                # 📂 Bước 1: Tải Dữ Liệu
                Chọn một trong hai cách tải dữ liệu bên dưới:
                """)

                data_info = gr.Markdown("""
                ### 💡 Hướng dẫn:
                - **Option 1** (Khuyến nghị): Upload 1 file Excel/CSV có cột 'Class' → Tự động chia train/test bằng SPXY
                - **Option 2**: Upload 2 files đã chia sẵn (train.xlsx + test.xlsx)
                - Sau khi xử lý xong → Kiểm tra biểu đồ phân bố dữ liệu
                """, elem_classes=["info-box"])

                with gr.Row():
                    with gr.Column(scale=1, elem_classes=["upload-section"]):
                        gr.Markdown("### 📄 Option 1: File Chưa Chia")
                        gr.Markdown("*Tự động chia train/test bằng thuật toán SPXY*")
                        single_file = gr.File(label="📎 Chọn file Excel hoặc CSV", file_types=['.xlsx', '.csv'], show_label=True, elem_classes=["file-upload"])

                        # Help panel for upload params
                        upload_help = gr.Markdown("💡 *Bấm tiêu đề xanh để xem giải thích chi tiết*", visible=True)

                        with gr.Row():
                            with gr.Column(scale=1):
                                btn_help_train_ratio = gr.Button("🎯 Tỷ lệ Train (Train Ratio)", elem_classes=["link-btn"])
                                train_ratio = gr.Slider(0.5, 0.9, value=0.7, step=0.05, label="", show_label=False)
                            with gr.Column(scale=1):
                                btn_help_random_seed = gr.Button("🎲 Random Seed", elem_classes=["link-btn"])
                                random_state = gr.Number(value=42, label="", precision=0, show_label=False)

                        hide_upload_help = gr.Button("Ẩn giải thích", variant="secondary", size="sm")
                        btn_single = gr.Button("▶️ Xử lý File", variant="primary", size="lg", elem_classes="pulse-btn")

                    with gr.Column(scale=1, elem_classes=["upload-section"]):
                        gr.Markdown("### 📂 Option 2: Files Đã Chia")
                        gr.Markdown("*Sử dụng train.xlsx và test.xlsx có sẵn*")
                        train_file = gr.File(label="📊 File Train", file_types=['.xlsx', '.csv'], show_label=True, elem_classes=["file-upload"])
                        test_file = gr.File(label="📋 File Test", file_types=['.xlsx', '.csv'], show_label=True, elem_classes=["file-upload"])
                        gr.Markdown("")  # Spacer
                        btn_split = gr.Button("▶️ Xử lý Files", variant="primary", size="lg", elem_classes="pulse-btn")

                # Thông tin kết quả và download
                gr.Markdown("---")
                gr.Markdown("## 📊 Kết Quả Xử Lý")
                data_result = gr.Markdown("*Kết quả sẽ hiển thị ở đây sau khi xử lý*", elem_classes=["status-message"])

                gr.Markdown("### 📈 Biểu Đồ Phân Bố Dữ Liệu")
                plot_image = gr.Image(label="Phân bố Train/Test", type="filepath", interactive=False, show_label=True)

                # Explanation functions for upload parameters
                def explain_train_ratio():
                    return (
                        """### 🎯 Tỷ lệ Train (Train Ratio)

                        - Tỷ lệ % dữ liệu dùng để huấn luyện model (phần còn lại để kiểm tra).
                        - **Khuyến nghị**: 0.7-0.8 (70-80%) cho hầu hết bài toán.
                        - Code dùng thuật toán **SPXY** để chia dữ liệu đồng đều.
                        """
                    )

                def explain_random_seed():
                    return (
                        """### 🎲 Random Seed là gì?

                        **Định nghĩa đơn giản:**
                        - Random Seed là 1 con số BẤT KỲ (42, 99, 5, 123, 9999... số nào cũng được!)
                        - Code dùng số này để **khởi tạo bộ trộn dữ liệu**
                        - Seed KHÁC → Code trộn dữ liệu theo cách KHÁC → Kết quả KHÁC

                        ### 📊 Ví dụ CỤ THỂ:

                        **Bạn có file 10 mẫu:** [A, B, C, D, E, F, G, H, I, J]

                        **Khi seed = 42:**
                        1. Code trộn thành: [D, A, H, B, I, C, J, E, F, G]
                        2. Lấy 70% = 7 mẫu đầu vào Train: [D, A, H, B, I, C, J]
                        3. Lấy 30% = 3 mẫu cuối vào Test: [E, F, G]
                        4. Chạy model → Test Accuracy = **93.2%**

                        **Khi seed = 99:**
                        1. Code trộn thành: [G, C, A, I, E, B, D, F, H, J]
                        2. Lấy 70% = 7 mẫu đầu vào Train: [G, C, A, I, E, B, D]
                        3. Lấy 30% = 3 mẫu cuối vào Test: [F, H, J]
                        4. Chạy model → Test Accuracy = **92.8%** (KHÁC vì test khác mẫu!)

                        ---

                        """
                    )

                # Wire click events for upload params
                btn_help_train_ratio.click(fn=explain_train_ratio, outputs=upload_help)
                btn_help_random_seed.click(fn=explain_random_seed, outputs=upload_help)
                hide_upload_help.click(fn=lambda: "💡 *Bấm tiêu đề xanh để xem giải thích chi tiết*", outputs=upload_help)

            # ===== TAB 2: TRAINING CONFIG =====
            with gr.Tab("⚙️ 2. Cấu Hình & Training"):
                training_info = gr.Markdown("ℹ️ Chưa train")

                gr.Markdown("### Thiết lập tham số training:")

                # Hộp hiện giải thích tham số khi bấm tiêu đề xanh + nút ẩn
                param_help = gr.Markdown("ℹ️ Bấm tiêu đề của tham số để xem giải thích.")
                hide_help_btn = gr.Button("❌ Ẩn giải thích", variant="secondary", size="sm")

                with gr.Row():
                    with gr.Column(scale=1):
                        btn_help_n_trials = gr.Button("🔢 N_TRIALS", elem_classes=["link-btn"])
                        n_trials = gr.Number(value=50, label="", precision=0, show_label=False, elem_classes=["training-input"])
                    with gr.Column(scale=1):
                        btn_help_cv_folds = gr.Button("📊 CV_FOLDS", elem_classes=["link-btn"])
                        cv_folds = gr.Number(value=5, label="", precision=0, show_label=False, elem_classes=["training-input"])

                with gr.Row():
                    with gr.Column(scale=1):
                        btn_help_optuna_timeout = gr.Button("⏱️ TIMEOUT (giây)", elem_classes=["link-btn"])
                        optuna_timeout = gr.Number(value=180, label="", precision=0, show_label=False, elem_classes=["training-input"])
                    with gr.Column(scale=1):
                        btn_help_output_dir = gr.Button("📁 OUTPUT_DIR", elem_classes=["link-btn"])
                        output_dir_name = gr.Textbox(value="ML_Output", label="", show_label=False, elem_classes=["training-input"])

                # Hàm giải thích các tham số
                def explain_n_trials():
                    return (
                        """### N_TRIALS là gì?

                        - Số lần thử để Optuna tìm bộ siêu tham số tốt nhất cho MỖI model.
                        - Mỗi trial = 1 bộ tham số khác nhau. Nhiều trials → dễ tìm cấu hình tốt hơn nhưng chạy lâu hơn.
                        - Gợi ý: 20–50 (nhanh, demo), 100–300 (chất lượng tốt), >300 (khi dữ liệu lớn và cần tối ưu kỹ).

                        Ví dụ:
                        - N_TRIALS = 50 → mỗi model sẽ thử 50 cấu hình khác nhau.
                        - 6 models × 50 trials = tối đa ~300 cấu hình được thử.
                        """
                    )

                def explain_cv_folds():
                    return (
                        """### CV_FOLDS là gì?

                        - Số phần (fold) khi chia tập train để đánh giá chéo (K-Fold CV).
                        - Lớn hơn → kết quả ổn định hơn nhưng tốn thời gian hơn. Phổ biến: 5 hoặc 10.

                        Ví dụ:
                        - CV_FOLDS = 5 → chia train thành 5 phần, luân phiên 5 lần train/validate, lấy điểm trung bình.
                        - Dữ liệu ít → nên dùng 5 để cân bằng tốc độ và độ ổn định.
                        """
                    )

                def explain_optuna_timeout():
                    return (
                        """### OPTUNA_TIMEOUT là gì?

                        - Thời gian tối đa (giây) để tối ưu MỖI model. Dù chưa hết N_TRIALS, Optuna sẽ dừng khi hết thời gian.
                        - Dùng để giới hạn tổng thời gian training.

                        Ví dụ:
                        - OPTUNA_TIMEOUT = 180 → mỗi model tối ưu tối đa 3 phút.
                        - Nếu N_TRIALS=200 nhưng 180 giây đã hết → dừng sớm, lấy cấu hình tốt nhất tìm được.
                        """
                    )

                def explain_output_dir():
                    return (
                        """### OUTPUT_DIR là gì?

                        - Tên thư mục lưu toàn bộ kết quả: results.xlsx, biểu đồ, và thư viện Arduino đã sinh.
                        - Có thể đặt theo project để dễ quản lý.

                        Ví dụ:
                        - OUTPUT_DIR = "ML_Output" → tạo thư mục ML_Output/ chứa toàn bộ kết quả.
                        - Mỗi lần chạy mới nên đổi tên để không ghi đè (vd: ML_Output_2025_12_29).
                        """
                    )

                # Gán sự kiện click để hiển thị/ẩn giải thích
                btn_help_n_trials.click(fn=explain_n_trials, outputs=param_help)
                btn_help_cv_folds.click(fn=explain_cv_folds, outputs=param_help)
                btn_help_optuna_timeout.click(fn=explain_optuna_timeout, outputs=param_help)
                btn_help_output_dir.click(fn=explain_output_dir, outputs=param_help)
                hide_help_btn.click(fn=lambda: "", outputs=param_help)

                gr.Markdown("---")
                gr.Markdown("### Chọn nhóm mô hình muốn train:")

                model_groups = gr.CheckboxGroup(
                    choices=[
                        "SVM (9 models)",
                        "Tree (5 models)",
                        "Neural Network (3 models)",
                        "KNN (2 models)",
                        "Discriminant (2 models)",
                        "Naive Bayes (2 models)"
                    ],
                    value=[
                        "SVM (9 models)",
                        "Tree (5 models)",
                        "Neural Network (3 models)",
                        "KNN (2 models)",
                        "Discriminant (2 models)",
                        "Naive Bayes (2 models)"
                    ],
                    label="Nhóm mô hình",
                    interactive=True
                )

                with gr.Row():
                    btn_select_all = gr.Button("✅ Chọn Tất Cả", size="sm")
                    btn_deselect_all = gr.Button("❌ Bỏ Chọn Tất Cả", size="sm")

                gr.Markdown("---")
                btn_train = gr.Button("🚀 Bắt Đầu Training", variant="primary", size="lg", elem_classes="pulse-btn")

                results_table = gr.Dataframe(
                    label="Kết quả Training",
                    interactive=False,
                    wrap=True
                )

            # ===== TAB 3: MODEL SELECTION =====
            with gr.Tab("🎯 3. Chọn Models"):
                gr.Markdown("""
                # 🎯 Bước 3: Chọn Models Tốt Nhất
                Sau khi training xong, chọn các models để tạo thư viện Arduino/ESP32:
                """)

                selection_info = gr.Markdown("*Kết quả sẽ hiển thị sau khi chọn models*", elem_classes=["status-message"])

                gr.Markdown("### 📋 Cách chọn models:")

                with gr.Row():
                    with gr.Column(scale=1):
                        selection_mode = gr.Radio(
                            choices=[
                                "Top N (Test Accuracy)",
                                "Top N (Train Accuracy)",
                                "Top N (Train Precision)",
                                "Top N (Test Precision)",
                                "Manual (nhập số)"
                            ],
                            value="Top N (Test Accuracy)",
                            label="🎯 Tiêu chí sắp xếp",
                            info="Chọn tiêu chí để lọc top N models"
                        )
                    with gr.Column(scale=1):
                        auto_count = gr.Number(
                            value=10,
                            label="📊 Số lượng models",
                            precision=0,
                            minimum=1,
                            maximum=50,
                            info="Chọn bao nhiêu models (mặc định 10)",
                            elem_classes=["training-input"]
                        )

                model_indices = gr.Textbox(
                    label="✏️ Nhập số thứ tự models (Manual mode)",
                    placeholder="Ví dụ: 1,3,5,7,10",
                    visible=False,
                    info="Nhập các số cách nhau bằng dấu phẩy"
                )

                models_list = gr.Markdown("ℹ️ Danh sách models sẽ hiện sau khi training", visible=False)

                btn_select = gr.Button("✅ Xác nhận chọn models", variant="primary", elem_classes="pulse-btn")

            # ===== TAB 4: ARDUINO LIBRARY =====
            with gr.Tab("📦 4. Tạo Thư Viện Arduino"):
                gr.Markdown("""
                # 📦 Bước 4: Tạo Thư Viện Arduino/ESP32
                Chuyển đổi models đã chọn thành code C/C++ để chạy trên vi điều khiển:
                """)

                generation_info = gr.Markdown("*Kết quả sẽ hiển thị sau khi tạo thư viện*", elem_classes=["status-message"])

                gr.Markdown("### ⚙️ Cấu hình thư viện:")

                with gr.Row():
                    with gr.Column(scale=1):
                        lib_name = gr.Textbox(value="MLPredictor", label="📚 Tên thư viện", info="Tên thư mục và class C++")
                    with gr.Column(scale=1):
                        separate_files = gr.Checkbox(
                            value=True,
                            label="✅ Tách mỗi model thành file riêng",
                            info="Khuyến nghị: Tối ưu Flash ROM cho ESP32/Arduino"
                        )

                gr.Markdown("---")
                btn_generate = gr.Button("🔨 Tạo Thư Viện Arduino", variant="primary", size="lg", elem_classes="pulse-btn")

            # ===== TAB 5: DOWNLOAD =====
            with gr.Tab("💾 5. Tải Xuống"):
                gr.Markdown("""
                # 💾 Bước 5: Tải Xuống Kết Quả
                Tất cả kết quả được đóng gói vào 1 file ZIP duy nhất:
                """)

                download_info = gr.Markdown("*Bấm nút bên dưới để tự động nén và tải xuống*", elem_classes=["status-message"])

                gr.Markdown("""
                ### 📦 Nội dung file ZIP:

                **File ZIP sẽ tự động nén và bao gồm:**
                - ✅ **train_data.xlsx & test_data.xlsx** - Dữ liệu đã chia
                - ✅ **data_distribution.png** - Biểu đồ phân bố dữ liệu
                - ✅ **results.xlsx** - TẤT CẢ kết quả (7 sheets):
                  - Training Results (đầy đủ kết quả số)
                  - Summary (kết quả format %)
                  - Library Info (tổng quan: tên, số model, kích thước Flash/SRAM, MCU đề xuất)
                  - Model Memory (Flash/SRAM từng model chi tiết)
                  - MCU Compatibility (tương thích 15 MCUs với % sử dụng)
                - ✅ **confusion_matrices/** - Tất cả ma trận nhầm lẫn (PNG)
                - ✅ **Thư viện Arduino hoàn chỉnh** (thư mục library với .h, .cpp, examples)

                **Lưu ý:** Bấm nút sẽ tự động nén và tải xuống ngay!
                """)

                download_file = gr.File(label="File ZIP kết quả")
                btn_download = gr.Button("📥 Tải Xuống Tất Cả (ZIP)", variant="primary", size="lg", elem_classes="pulse-btn")

        # ===== EVENT HANDLERS =====

        # Chọn/Bỏ chọn tất cả nhóm models
        btn_select_all.click(
            lambda: [
                "SVM (9 models)",
                "Tree (5 models)",
                "Neural Network (3 models)",
                "KNN (2 models)",
                "Discriminant (2 models)",
                "Naive Bayes (2 models)"
            ],
            outputs=[model_groups]
        )

        btn_deselect_all.click(
            lambda: [],
            outputs=[model_groups]
        )

        # Upload single file
        btn_single.click(
            gradio_process_single_file,
            inputs=[single_file, train_ratio, random_state],
            outputs=[data_result, plot_image]
        )

        # Upload split files
        btn_split.click(
            gradio_process_split_files,
            inputs=[train_file, test_file],
            outputs=[data_result, plot_image]
        )

        # Train models
        btn_train.click(
            gradio_train_models,
            inputs=[n_trials, cv_folds, optuna_timeout, output_dir_name, model_groups],
            outputs=[training_info, results_table, models_list]
        )

        # Toggle visibility cho manual mode
        def toggle_visibility(mode):
            if mode == "Manual (nhập số)":
                return gr.update(visible=True), gr.update(visible=True)
            else:
                return gr.update(visible=False), gr.update(visible=False)

        selection_mode.change(
            toggle_visibility,
            inputs=[selection_mode],
            outputs=[model_indices, models_list]
        )

        # Select models
        btn_select.click(
            gradio_select_models,
            inputs=[selection_mode, model_indices, auto_count],
            outputs=[selection_info, generation_info]
        )

        # Generate Arduino library
        btn_generate.click(
            gradio_generate_arduino,
            inputs=[lib_name, separate_files],
            outputs=[generation_info, download_info]
        )

        # Create download package
        btn_download.click(
            gradio_create_download,
            outputs=[download_file, download_info]
        )

    return app

# ============================================================================
# MAIN EXECUTION
# ============================================================================

def main():
    """Hàm chính thực thi toàn bộ pipeline"""
    print("\n" + "=" * 80)
    print(" " * 20 + "MACHINE LEARNING PIPELINE")
    print(" " * 15 + "SVM Models for Arduino/ESP32")
    print("=" * 80)
    print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    try:
        # 1. Load dữ liệu
        X_train, X_test, y_train, y_test, feature_names = load_data()

        # 2. Vẽ biểu đồ phân bố
        plot_data_distribution(X_train, X_test, y_train, y_test, feature_names)

        # 3. Train models
        all_results, trained_models, confusion_matrices = train_all_models(
            X_train, y_train, X_test, y_test
        )

        # 4. Lưu kết quả
        save_results(all_results, confusion_matrices)

        # 5. Chọn models để generate Arduino library
        selected_models = select_models_for_arduino(all_results, trained_models)

        # 6. Tạo thư viện Arduino (chỉ cho models được chọn)
        class_labels = sorted(np.unique(y_train))
        generate_arduino_library(selected_models, feature_names, class_labels, X_train, X_test)

        print("\n" + "=" * 80)
        print(" " * 25 + "HOÀN THÀNH!")
        print("=" * 80)
        print(f"Finished at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"\nKết quả đã được lưu tại: {OUTPUT_DIR}/")
        print("\nCác file output:")
        print("  ✓ train_data.xlsx, test_data.xlsx - Dữ liệu đã chia")
        print("  ✓ data_distribution.png - Biểu đồ phân bố")
        print("  ✓ results.xlsx - Kết quả training (7 sheets: Results, Summary, Library Info, Model Memory, MCU Compatibility)")
        print("  ✓ confusion_matrices/ - Ma trận nhầm lẫn")
        print(f"  ✓ {ARDUINO_LIB_NAME}/ - Thư viện Arduino/ESP32")
        print("\n💡 Tip: Chạy package_and_download() nếu cần tạo file ZIP")

    except Exception as e:
        print(f"\n❌ LỖI: {e}")
        import traceback
        traceback.print_exc()

# ============================================================================
# RUN
# ============================================================================

if __name__ == "__main__":
    # Nếu chạy trên Colab/Kaggle, tự động dùng Gradio
    if IN_CLOUD:
        print("\n" + "=" * 60)
        print(f"🌐 {'Google Colab' if IN_COLAB else 'Kaggle'} - ML Pipeline for Arduino/ESP32")
        print("=" * 60)

        if not GRADIO_AVAILABLE:
            print("📦 Đang cài đặt Gradio...")
            import subprocess
            subprocess.run([sys.executable, "-m", "pip", "install", "-q", "gradio"])
            import gradio as gr
            GRADIO_AVAILABLE = True

        print("\n🚀 Đang khởi động giao diện web...\n")

        app = create_gradio_interface()

        print("\n" + "=" * 60)
        print("✅ NHẤN VÀO LINK BÊN DƯỚI ĐỂ MỞ GIAO DIỆN:")
        print("=" * 60 + "\n")

        app.launch(
            share=True,
            debug=False,
            show_error=True,
            max_file_size="100mb"
        )

    else:
        # Local machine - cho phép chọn mode
        import argparse
        parser = argparse.ArgumentParser(description='ML Pipeline - Train models and generate Arduino libraries')
        parser.add_argument('--mode', type=str, default='gradio', choices=['cli', 'gradio'],
                           help='Chế độ chạy: cli (command line) hoặc gradio (web interface)')
        parser.add_argument('--share', action='store_true', help='Tạo public link cho Gradio (share=True)')
        parser.add_argument('--port', type=int, default=7860, help='Port cho Gradio server')

        args = parser.parse_args()

        if args.mode == 'gradio':
            # Chế độ Gradio Web Interface
            if not GRADIO_AVAILABLE:
                print("❌ Gradio chưa được cài đặt!")
                print("Cài đặt bằng: pip install gradio")
                print("Hoặc chạy ở chế độ CLI: python generate11.py --mode cli")
                sys.exit(1)

            print("\n" + "=" * 80)
            print(" " * 20 + "🌐 GRADIO WEB INTERFACE")
            print(" " * 15 + "ML Pipeline for Arduino/ESP32")
            print("=" * 80)
            print(f"\n✅ Khởi động Gradio server trên port {args.port}...")

            app = create_gradio_interface()
            app.launch(
                share=args.share,
                server_name="0.0.0.0",
                server_port=args.port,
                show_error=True
            )

        else:
            # Chế độ CLI (Command Line) - code gốc
            # Mount Google Drive nếu cần
            if IN_COLAB and (SINGLE_FILE_PATH.startswith('/content/drive') or
                             TRAIN_FILE_PATH.startswith('/content/drive') or
                             TEST_FILE_PATH.startswith('/content/drive')):
                print("Mounting Google Drive...")
                drive.mount('/content/drive')

            # Chạy pipeline CLI
            main()

